In [2]:
# 31319413,19752556中发现"ALZHEI-MER'S DISEASE"划分到了'Parkinson Disease'，需要处理！

In [3]:
pwd

'/root/hald_autoupdate/2022_12_09'

In [95]:
import pickle
import json
import time
import datetime
import copy

In [5]:
now=datetime.datetime.now()

In [6]:
nowtime=now.strftime('%Y/%m/%d')

In [7]:
now_time=now.strftime('%Y_%m_%d')

In [8]:
print(nowtime)

2023/02/25


In [9]:
time_without_day=now.strftime('%Y/%m')

In [10]:
time_without_day

'2023/02'

In [11]:
### today 2022_07 FORMAT
### now.strftime('%Y_%m')

In [12]:
### 2022/07/30 FORMAT
def cal_time(time):
    time = time.split('/')
    year = int(time[0])
    month = int(time[1]) - 3
    day = int(time[2])
    # 防止月份出界
    if month < 1:
        month = 12
        year -= 1
    time = datetime.datetime(year=year, month=month, day=1).strftime('%Y/%m/%d')
    return time

In [13]:
### 2022_07_30 FORMAT
def cal_time2(time):
    time = time.split('/')
    year = int(time[0])
    month = int(time[1]) - 3
    day = int(time[2])
    # 防止月份出界
    if month < 1:
        month = 12
        year -= 1
    time = datetime.datetime(year=year, month=month, day=1).strftime('%Y_%m_%d')
    return time

In [14]:
### 2022_07 FORMAT
def cal_time3(time):
    time = time.split('/')
    year = int(time[0])
    month = int(time[1]) - 3
    day = int(time[2])
    # 防止月份出界
    if month < 1:
        month = 12
        year -= 1
    time = datetime.datetime(year=year, month=month, day=1).strftime('%Y_%m')
    return time

In [15]:
former_time=cal_time(nowtime)

In [16]:
time

<module 'time' (built-in)>

In [17]:
former_time

'2022/12/01'

In [18]:
former_time='2022/04/01'

now_time='2022_12_09'

In [19]:
import pymongo 

In [20]:
from pymongo import MongoClient

In [21]:
client = MongoClient(username="hald",password="ddd")

In [22]:
db = client.hald

In [355]:
raw_pick=pickle.load(open('raw_pick.pkl','rb'))

In [356]:
raw_pick_pubtator=pickle.load(open('raw_pick_pubtator.pkl','rb'))

In [357]:
pubtator_raw=pickle.load(open('pubtator_raw.pkl','rb'))

In [358]:
pubtator_raw_dict2AB={}
for i in pubtator_raw:
    pubtator_raw_dict2AB[i.get('id')]=i.get('AB')

# Gene_raw

In [25]:
collections_Gene_raw=db.Gene_raw

In [359]:
rawpickwithAB={}
for i in raw_pick_pubtator:
    rawpickwithAB[i.get('PMID')]=i

In [360]:
pubtator_gene=[]
for i in pubtator_raw:
    if i.get('type')=='Gene':
        pubtator_gene.append(i)

In [361]:
#7246
len(pubtator_gene)

7246

In [362]:
import pandas as pd
import json
import re

In [363]:
Gene2ncbi=pd.read_csv('../ncbi/Gene_ncbi.txt',sep='\t',encoding='utf-8',converters = {'NCBI Gene ID(supplied by NCBI)':str})

In [364]:
ncbi2Gene_dict=dict(zip(Gene2ncbi['NCBI Gene ID(supplied by NCBI)'],Gene2ncbi['Approved symbol'] ))

In [365]:
error=[]
pubtator_genes=[]
for i in pubtator_gene:
    if ncbi2Gene_dict.get(i.get('identifier')):
        i['entity']=ncbi2Gene_dict.get(i.get('identifier'))
        pubtator_genes.append(i)
    else:
        error.append(i)

In [366]:
len(pubtator_genes)

5899

In [367]:
Genelist=pickle.load(open('../dict/Genedict_list.pkl','rb'))

In [368]:
Gene_list={}
for i in Genelist:
    Gene_list[i.lower()]=i.lower()

In [369]:
filtered_pubtator_genes=[]
drop_genes=[]
for i in pubtator_genes:
    if Gene_list.get(i.get('text').lower()):
        filtered_pubtator_genes.append(i)
    else:
        drop_genes.append(i)

In [370]:
len(drop_genes)

1361

In [371]:
len(filtered_pubtator_genes)

4538

In [372]:
filtered_pubtator_genes[10]

{'id': '36466095',
 'AB': 'Ovarian aging is associated with a decrease in fecundity. Increased oxidative stress of granulosa cells (GCs) is an important contributor. We thus asked whether there is an oxidative stress-related gene signature in GCs associated with ovarian aging. Public nonhuman primate (NHP) single-cell transcriptome was processed to identify GC cluster. Then, a GC signature for ovarian aging was established based on six oxidative stress-related differentially expressed genes (MAPK1, STK24, AREG, ATG7, ANXA1, and PON2). Receiver operating characteristic (ROC) analysis confirmed good discriminating capacity in both NHP single-cell and human bulk transcriptome datasets. Gene expression levels were investigated using qPCR in the human ovarian granulosa-like tumor cell line (KGN) and mouse GCs. In an oxidative stress model, KGN cells were treated with menadione (7.5 muM, 24 h) to induce oxidative stress, after which upregulation of MAPK1, STK24, ATG7, ANXA1, and PON2 and dow

In [373]:
for i in filtered_pubtator_genes:
    i['ner']='HALD_Gene'
    i['PMID']=i.get('id')
    i['word']=i.get('text')

In [374]:
import spacy
import scispacy

import pandas as pd
import requests 
from collections import Counter
from tqdm.notebook import tqdm
tqdm.pandas()
import json
import ast
from math import ceil
from itertools import combinations, permutations
import en_ner_bc5cdr_md, en_ner_bionlp13cg_md, en_ner_jnlpba_md
from spacy.tokenizer import Tokenizer
import string
from itertools import permutations

/root/miniconda3/envs/hald/lib/python3.8/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [375]:
ner_bc5c = en_ner_bc5cdr_md.load()
ner_bionl = en_ner_bionlp13cg_md.load()
ner_jnlp = en_ner_jnlpba_md.load()

In [376]:
def get_entities(ti, model, keep=[]):
    doc = model(ti)
    if len(keep) > 0:
        return [(x.text.replace('\n', ''), x.label_) for x in doc.ents if str(x.label_) in keep]
    return [(x.text.replace('\n', ''), x.label_) for x in doc.ents ]

In [377]:
raw_pick_pubtator_pmid=[]
for i in raw_pick_pubtator:
    if i.get('AB'):
        raw_pick_pubtator_pmid.append(i.get('PMID'))
    else:
        pass

In [378]:
filtered_pubtator_genes[0]

{'id': '36472108',
 'AB': "The title usage of Unde venisti 'from where have you come' is from a now dead language (Latin) that foundationally influenced modern English (not the major influence, but an essential formative one). This is an apt analogy for how both the ancient eukaryotic and eumetazoan functions of PGRMC proteins (PGRMC1 and PGRMC2 in mammals) probably influence modern human biology: via a formative trajectory from an evolutionarily foundational fulcrum. There is an arguable probability, although not a certainty, that PGRMC-like proteins were involved in eukaryogenesis. If so, then the proto-eukaryotic ancestral protein is modelled as having initiated the oxygen-induced and CYP450 (Cytochrome P450)-mediated synthesis of sterols in the endoplasmic reticulum to regulate proto-mitochondrial activity and heme homeostasis, as well as having enabled sterol transport between endoplasmic reticulum (ER) and mitochondria membranes involving the actin cytoskeleton, transport of heme

prepare_for_scispacy={}
for i in filtered_pubtator_genes:
    if not filtered_pubtator_genes.get(i.get('PMID')):
        filtered_pubtator_genes[i.get('PMID')]=i

In [379]:
# 186s
import datetime
import re
from multiprocessing import Process, Manager, Pool
import os, time, random
def work(i,finallist_tags,lock):
    # lock.acquire()
    finallist_tags.append({'PMID':i.get('PMID'), 'AB':i.get('AB'), 'sentence':i.get('sentence'), 'sentenceIndex': i.get('sentenceIndex'),\
                          'sentenceSize':i.get('sentenceSize'),'word': i.get('word'),'identifier':i.get('identifier'),'type':i.get('type'),\
                           'entity':i.get('entity'),'jnlpba_tags':get_entities(i.get('sentence'), model=ner_jnlp, keep=['DNA','RNA','PROTEIN']),\
                          'bc5cdr_tags':get_entities(i.get('sentence'), model=ner_bc5c),\
                          'bionlp13cg_tags':get_entities(i.get('sentence'), model=ner_bionl, \
                          keep=['GENE_OR_GENE_PRODUCT'])})
    # lock.release()
    # bc5cdr_tags: DISEASE, CHEMICAL
    
if __name__=='__main__':
    start=datetime.datetime.now()
    print('Parent process %s.' % os.getpid())
    p=Pool(60)
    manager=Manager()
    finallist_tags=manager.list()
    lock=manager.Lock()
    for i in filtered_pubtator_genes:
        p.apply_async(work, args=(i,finallist_tags,lock))

    print('Waiting for all subprocesses done...')
    
    p.close()
    p.join()
    print('All subprocesses done.')

    end=datetime.datetime.now()
    print("The running time is "+str((end-start).seconds)+"s")  

Parent process 99086.
Waiting for all subprocesses done...
All subprocesses done.
The running time is 45s


In [380]:
Gene_tags=list(finallist_tags)

In [381]:
#4538
len(Gene_tags)

4538

In [382]:
import copy

In [383]:
scispicy_Gene=copy.deepcopy(Gene_tags)

In [384]:
for i in scispicy_Gene:
    i['ScispaCy']=[]
    if i.get('jnlpba_tags'):
        for j in i.get('jnlpba_tags'):
            if j[1]=='DNA' or j[1]=='PROTEIN':
                i['ScispaCy']+=[j[0]]
    if i.get('bionlp13cg_tags'):
        for k in i.get('bionlp13cg_tags'):
            i['ScispaCy']+=[k[0]]
    i['ScispaCy']=list(set(i.get('ScispaCy')))

In [386]:
scicpacy_Genes=[]
not_scicpacy_Genes=[]
for i in scispicy_Gene:
    if i.get('ScispaCy'):
        scicpacy_Genes.append(i)
    else:
        not_scicpacy_Genes.append(i)

In [387]:
# 4123
len(scicpacy_Genes)

4123

In [392]:
len(not_scicpacy_Genes)

415

In [397]:
not_scicpacy=[]
for i in not_scicpacy_Genes:
    not_scicpacy.append({'PMID':i.get('PMID'),'word':i.get('word'),'sentence':i.get('sentence')})

In [399]:
not_scicpacy[-10:-1]

[{'PMID': '35181743',
  'word': 'Fus1',
  'sentence': 'They suggest that Fus1-dependent mechanisms are relevant in etiologies of diseases beyond cancer, such as chronic inflammation, bacterial and viral infections, premature aging, and geriatric diseases.'},
 {'PMID': '35181743',
  'word': 'Fus1',
  'sentence': 'This Fus1-mediated cellular homeostasis is at the crux of tumor suppressor, anti-inflammatory and anti-aging activities.'},
 {'PMID': '35170409',
  'word': 'CD8',
  'sentence': 'Telomere length of naive (CD45RA+) and memory (CD45RO+) CD8 T cells was quantified by real-time PCR.'},
 {'PMID': '35202044',
  'word': 'TN',
  'sentence': 'As chronic pain disorders can differ in etiology, severity, pain frequency, and sex-linked prevalence, we hypothesize that the expression of brain-AGE may be pain specific and differ between discrete chronic pain disorders.We built a machine learning model using T1-weighted anatomical MRI from 812 healthy controls to extract brain-AGE for 45 trigemi

In [388]:
scicpacy_Genes_final=[]
drop_scispacy=[]
for i in scicpacy_Genes:
    pattern=i.get('word').translate(str.maketrans({"-":  r"\-","\\": r"\\", "^":  r"\^","$":  r"\$","*":  r"\*",".":  r"\.","(":  r"\(",")":  r"\)","+":  r"\+","[":  r"\[","]":  r"\]",
                              "{":  r"\{","}":  r"\}","|":  r"\|","?":  r"\?"}))
    for j in i.get('ScispaCy'):
        if i.get('word')==j or re.search(pattern, j):
            scicpacy_Genes_final.append(i)
            break
    else:
        drop_scispacy.append(i)

In [389]:
# 3701
len(scicpacy_Genes_final)

3701

In [396]:
for i in scicpacy_Genes_final:
    if re.search('-',i.get('word')):
        print(i)
        break

{'PMID': '36460954', 'AB': 'BACKGROUND: Cardiovascular disease was the most common disease among the elderly with high morbidity and mortality. Circ_0004104 was demonstrated to be involved in the regulation of atherosclerosis. METHODS: Quantitative real-time polymerase chain reaction was employed to measure the expression of circ_0004104, miR-942-5p and Rho associated coiled-coil containing protein kinase 2 (ROCK2). Cell proliferation was tested by 3-(4,5-dimethyl-2-thiazolyl)-2,5-diphenyl-2-H-tetrazolium bromide (MTT) assay. Cell apoptosis was measured by flow cytometry, and tube formation assay was used to detect the angiogenesis ability of cells. Western blot assay was performed to assess protein levels. Enzyme-linked immunosorbent assay was used to detect the release of IL-1beta and TNF-alpha. The relationship between miR-942-5p and circ_0004104 or ROCK2 was identified by dual-luciferase reporter assay, RNA immunoprecipitation (RIP) assay, and RNA pull-down assay. RESULTS: Oxidized

In [390]:
scicpacy_Genes_final[0]

{'PMID': '36472108',
 'AB': "The title usage of Unde venisti 'from where have you come' is from a now dead language (Latin) that foundationally influenced modern English (not the major influence, but an essential formative one). This is an apt analogy for how both the ancient eukaryotic and eumetazoan functions of PGRMC proteins (PGRMC1 and PGRMC2 in mammals) probably influence modern human biology: via a formative trajectory from an evolutionarily foundational fulcrum. There is an arguable probability, although not a certainty, that PGRMC-like proteins were involved in eukaryogenesis. If so, then the proto-eukaryotic ancestral protein is modelled as having initiated the oxygen-induced and CYP450 (Cytochrome P450)-mediated synthesis of sterols in the endoplasmic reticulum to regulate proto-mitochondrial activity and heme homeostasis, as well as having enabled sterol transport between endoplasmic reticulum (ER) and mitochondria membranes involving the actin cytoskeleton, transport of he

In [391]:
len(drop_scispacy)

422

In [58]:
scicpacy_pmids=[]
for i in scicpacy_Genes_final:
    if i.get('PMID') not in scicpacy_pmids:
        scicpacy_pmids.append(i.get('PMID'))

In [59]:
len(scicpacy_pmids)

792

In [60]:
def query_raw_id(ids, url="https://bern.korea.ac.kr/pubmed/"):
    return requests.get(url + ','.join(ids), verify=False).json()

now_time='2022_12_01'

In [61]:
os.makedirs('../bern/'+now_time)

In [62]:
f=open('../bern/'+now_time+'/download.txt','a')
bern_download_id=1
for j in range(0,len(scicpacy_pmids),5):
    f.write('wget https://bern.korea.ac.kr/pubmed/'+','.join(scicpacy_pmids[j:j+5])+' --no-check-certificate -O '+'../bern/'+now_time+'/bern'+str(bern_download_id)+'.json \n')
    bern_download_id+=1
f.close()

In [63]:
f=open('../bern/'+now_time+'/run.sh','w')
f.write("while read rows; do $rows  ; done < ../bern/"+now_time+'/download.txt')
f.close()

In [64]:
print("nohup bash "+'../bern/'+now_time+'/run.sh > '+'../bern/'+now.strftime('%Y_%m_%d')+'/bern.log &')

nohup bash ../bern/2022_12_09/run.sh > ../bern/2022_12_09/bern.log &


## RUN BERN ON LINUX!!!!!!

In [66]:
'''
%%capture
os.system("bash "+'../bern/'+now.strftime('%Y_%m_%d')+'/run.sh > '+'../bern/'+now.strftime('%Y_%m_%d')+'/bern.log')
'''

'\n%%capture\nos.system("bash "+\'../bern/\'+now.strftime(\'%Y_%m_%d\')+\'/run.sh > \'+\'../bern/\'+now.strftime(\'%Y_%m_%d\')+\'/bern.log\')\n'

In [67]:
pwd

'/root/hald_autoupdate/2022_12_05'

In [61]:
all_json=[]
for i in os.listdir('../bern/'+now_time+'/'):
    if i.split('.')[-1] == 'json':
        all_json+=json.load(open('../bern/'+now_time+'/'+i))

In [62]:
allbern=copy.deepcopy(all_json)

In [63]:
len(allbern)

792

In [64]:
bern_json={}
for i in allbern:
    bern_value=[]
    if i.get('denotations'):
        for j in i.get('denotations'):
            if j.get('obj')=='gene' or j.get('obj')=='protein':
                bern_value.append(i.get('text')[j.get('span').get('begin'):j.get('span').get('end')])
    bern_json[i.get('sourceid')]=bern_value    

In [65]:
len(bern_json)

792

In [66]:
count=0
filtered_bern_json={}
for i,j in bern_json.items():
    if j:
        count+=1
        filtered_bern_json[i]=j
print(count)

766


In [67]:
list(filtered_bern_json.items())[0]

('36053756',
 ['amyloid beta',
  'Abeta',
  'tau',
  'Abeta',
  'Abeta',
  'mitochondrial 18 kDa translocator protein',
  'TSPO'])

In [68]:
SciSpacy_raw=copy.deepcopy(scicpacy_Genes_final)

In [69]:
len(SciSpacy_raw)

3701

In [70]:
SciSpacy_raw_dict={}
for i in SciSpacy_raw:
    SciSpacy_raw_dict[i.get('PMID')]=SciSpacy_raw_dict.get(i.get('PMID'),[])+[i]

In [71]:
len(SciSpacy_raw_dict)

792

In [72]:
cs=0
intersec={}
dropout=[]
for i,j in filtered_bern_json.items():
    if SciSpacy_raw_dict.get(i):
        midd=[]
        for k in SciSpacy_raw_dict.get(i):
            if k.get('word') in j:
                k['bern']=j
                midd.append(k)
            else:
                for m in j:
                    pattern=k.get('word').translate(str.maketrans({"-":  r"\-","\\": r"\\", "^":  r"\^","$":  r"\$","*":  r"\*",".":  r"\.","(":  r"\(",")":  r"\)","+":  r"\+","[":  r"\[","]":  r"\]",
                      "{":  r"\{","}":  r"\}","|":  r"\|","?":  r"\?"}))
                    if re.search(pattern, m):
                        k['bern']=j
                        midd.append(k)
                        break
                else:
                    dropout.append(k)
        intersec[i]=midd
        

In [73]:
len(intersec)

766

In [74]:
len(dropout)

59

In [75]:
final_Gene_raw=[]
for i in intersec.values():
    final_Gene_raw+=i

In [76]:
len(final_Gene_raw)

3580

In [77]:
Genes_inte=copy.deepcopy(final_Gene_raw)

In [78]:
Gene2blank={}
for i in Genes_inte:
    Gene2blank[i.get('entity')]=''

In [79]:
len(Gene2blank)

741

In [80]:
Gene2ncbi=pd.read_csv('../ncbi/Gene_ncbi.txt',sep='\t',encoding='utf-8',converters = {'NCBI Gene ID(supplied by NCBI)':str})

In [81]:
Gene2ncbi_dict=dict(zip(Gene2ncbi['Approved symbol'], Gene2ncbi['NCBI Gene ID(supplied by NCBI)']))

In [183]:
Gene_converter=pickle.load(open('../ncbi/Gene_converter.pkl','rb'))

In [83]:
len(Gene_converter)

4854

In [84]:
Genes_inte_copy=copy.deepcopy(Genes_inte)

In [290]:
rawpickwithAB={}
for raw in raw_pick_pubtator:
    if raw.get('AB'):
        rawpickwithAB[raw.get('PMID')]=raw

In [86]:
Gene_deduplicated=set()
Gene_deduplicated_count=0
Gene_list_raw=[]

for i in Genes_inte_copy:
    k=i.get('PMID')
    i['DP']=rawpickwithAB.get(k).get('DP')
    i['AB']=rawpickwithAB.get(k).get('AB')
    i['JT']=rawpickwithAB.get(k).get('JT')
    i['URL']='https://pubmed.ncbi.nlm.nih.gov/'+k
    i['TI']=rawpickwithAB.get(k).get('TI')
    i['target']=i.get('word')
    i['type']='Gene'
    tuples=(i.get("PMID"),i.get("entity"),i.get("word"),i.get("sentence"))
    if tuples not in Gene_deduplicated:
        Gene_deduplicated.add(tuples)
        Gene_list_raw.append(i)
    else:
        Gene_deduplicated_count+=1
        continue

In [87]:
len(Gene_list_raw)

3216

In [464]:
Gene2blank={}
for i in Gene_list_raw:
    Gene2blank[i.get('entity')]=''

In [465]:
len(Gene2blank)

741

In [90]:
import time
import datetime
from multiprocessing import Process, Manager, Pool
import os, time, random
from lxml import etree
import requests
def get_RNA_info(i,final_RNAs_inte):
    ncbiid=RNA2ncbi_dict.get(i)
    ncbiurl='https://www.ncbi.nlm.nih.gov/gene/' + ncbiid
    headers = {
        "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 \
        (KHTML, like Gecko) Chrome/83.0.4103.97 Safari/537.36"
    }
    url = 'https://www.ncbi.nlm.nih.gov/gene/'+ncbiid
    
    res=requests.get(url=url,headers=headers)
    html = etree.HTML(res.content)
    ncbisummary={
    # 'numbersOfLiterature' : len(i.get('PMID')),
    'Official_Symbol' : html.xpath('//*[@id="summaryDl"]/dt[contains(.//text(),"Official")]/following::dd[1]/text()')[0],
    'Official_Full_Name' : html.xpath('//*[@id="summaryDl"]/dt[contains(.//text(),"Official")]/following::dd[1]/text()')[1],
    # 'Also_known_as' : ''.join(html.xpath('//*[@id="summaryDl"]/dt[contains(.//text(),"Also known as")]/following::dd[1]/text()')).replace('HGNC:HGNC:',''),
    'HGNC_ID' : ''.join(html.xpath('//*[@id="summaryDl"]/dt[contains(.//text(),"Primary source")]/following::dd[1]/a/text()')),
    'HGNC_URL' : ''.join(html.xpath('//*[@id="summaryDl"]/dt[contains(.//text(),"Primary source")]/following::dd[1]/a/@href')),
    'Ensembl_ID' :  ''.join(html.xpath('//*[@id="summaryDl"]/dt[contains(.//text(),"See related")]/following::dd[1]/a[1]/text()')),
    'Ensembl_URL' : ''.join(html.xpath('//*[@id="summaryDl"]/dt[contains(.//text(),"See related")]/following::dd[1]/a[1]/@href')),
    'MIM_ID' : ''.join(html.xpath('//*[@id="summaryDl"]/dt[contains(.//text(),"See related")]/following::dd[1]/a[2]/text()')),
    'MIM_URL' : ''.join(html.xpath('//*[@id="summaryDl"]/dt[contains(.//text(),"See related")]/following::dd[1]/a[2]/@href')),
    'AllianceGenome_ID' : ''.join(html.xpath('//*[@id="summaryDl"]/dt[contains(.//text(),"See related")]/following::dd[1]/a[3]/text()')),
    'AllianceGenome_URL' : ''.join(html.xpath('//*[@id="summaryDl"]/dt[contains(.//text(),"See related")]/following::dd[1]/a[3]/@href')),
    'NCBI_ID' : ncbiid ,
    'NCBI_URL' : ncbiurl
    }
    try:
        also=''.join(html.xpath('//*[@id="summaryDl"]/dt[contains(.//text(),"Also known as")]/following::dd[1]/text()')).replace('HGNC:HGNC:','')
    except:
        also=''
    finally:
        ncbisummary.update({'Also_known_as' : also})
    try:
        also2=''.join(html.xpath('//*[@id="summaryDl"]/dt[contains(.//text(),"Summary")]/following::dd[1]/text()'))
    except:
        also2=''
    finally:
        ncbisummary.update({'Summary' : also2})
    final_RNAs_inte[i]=ncbisummary
    # print(ncbisummary)
    time.sleep(0.5)

In [91]:
judge=False
p = Pool(10)
manager=Manager()
final_Genes_inte = manager.dict()
for i in Gene2blank.keys():
    if not Gene_converter.get(i):
        p.apply_async(get_Gene_info, args=(i,final_Genes_inte))
        judge=True
p.close()
p.join()
if judge:
    Gene_converter.update(dict(final_Genes_inte))
    pickle.dump(Gene_converter, open('../ncbi/Gene_converter.pkl','wb'))

In [468]:
# 4854
len(Gene_converter)

4854

In [560]:
Gene_converter.get('CLPS')

{'Official_Symbol': 'CLPS',
 'Official_Full_Name': 'colipase',
 'HGNC_ID': 'HGNC:HGNC:2085',
 'HGNC_URL': 'https://www.genenames.org/data/gene-symbol-report/#!/hgnc_id/HGNC:2085',
 'Ensembl_ID': 'Ensembl:ENSG00000137392',
 'Ensembl_URL': 'http://www.ensembl.org/id/ENSG00000137392',
 'MIM_ID': 'MIM:120105;',
 'MIM_URL': 'https://www.ncbi.nlm.nih.gov/omim/120105',
 'AllianceGenome_ID': 'AllianceGenome:HGNC:2085',
 'AllianceGenome_URL': 'https://www.alliancegenome.org/gene/HGNC:2085',
 'NCBI_ID': '1208',
 'NCBI_URL': 'https://www.ncbi.nlm.nih.gov/gene/1208',
 'Also_known_as': '',
 'Summary': 'The protein encoded by this gene is a cofactor needed by pancreatic lipase for efficient dietary lipid hydrolysis. It binds to the C-terminal, non-catalytic domain of lipase, thereby stabilizing an active conformation and considerably increasing the overall hydrophobic binding site. The gene product allows lipase to anchor noncovalently to the surface of lipid micelles, counteracting the destabilizin

In [469]:
for i in range(len(Gene_list_raw)):
    Gene_list_raw[i]=dict(Gene_list_raw[i],**Gene_converter.get(Gene_list_raw[i].get('entity')))

In [470]:
import time
import datetime
import re
dropout=[]
months=['Jan','Feb','Mar','Apr','May','Jun','Jul','Aug','Sep','Oct','Nov','Dec']
import re

for i in Gene_list_raw:
    j=i.get('DP')
    if re.search('[S,s]upplement',j,flags=re.M|re.I):
        j=re.sub('[S,s]upplement.*','',j,flags=re.M|re.I)
    j=re.sub('\s?-\s?','-',j)
    j=re.sub('\.','',j)
    j=re.sub('Aout','Aug',j)
    if len(j.split(' '))==3:
        try:
            date=int(datetime.datetime.strftime(datetime.datetime.strptime(j,'%Y %b %d'), '%Y%m%d'))
        except:
            if j.split()[1] in months:
                if re.search(r'\d+',j.split()[2],flags=re.M|re.I):
                    j=j.split()[0]+' '+j.split()[1]+' '+ re.search(r'\d+',j.split()[2],flags=re.M|re.I).group()
                    try:
                        date=int(datetime.datetime.strftime(datetime.datetime.strptime(j,'%Y %b %d'), '%Y%m%d'))
                    except:
                        j=j[:4]
                        date=int(datetime.datetime.strftime(datetime.datetime.strptime(j,'%Y'), '%Y'+'0101'))
                        print('***************************************ERROR**********************************************')
                else:
                    j=j.split()[0]+' '+j.split()[1]+' '+ '1'
                    try:
                        date=int(datetime.datetime.strftime(datetime.datetime.strptime(j,'%Y %b %d'), '%Y%m%d'))
                    except:
                        print(j)
                        j=j[:4]
                        date=int(datetime.datetime.strftime(datetime.datetime.strptime(j,'%Y'), '%Y'+'0101'))
            elif j.split()[1] not in months:
                if re.search(r'([a-zA-Z]+)[-/]([a-zA-Z]+)',j.split()[1],flags=re.M|re.I) or re.search(r'([a-zA-Z]+)[-/](\d+)',j.split()[1],flags=re.M|re.I):
                    if re.search(r'([a-zA-Z]+)[-/]([a-zA-Z]+)',j.split()[1],flags=re.M|re.I):
                        middle=re.search(r'([a-zA-Z]+)[-/]([a-zA-Z]+)',j.split()[1],flags=re.M|re.I).group(1)
                    else:
                        middle=re.search(r'([a-zA-Z]+)[-/](\d+)',j.split()[1],flags=re.M|re.I).group(1)
                    if middle in months:
                        if re.search(r'\d+',j.split()[2],flags=re.M|re.I):
                            j=j.split()[0]+' '+middle+' '+ re.search(r'\d+',j.split()[2],flags=re.M|re.I).group()
                        else:
                            j=j.split()[0]+' '+middle+' '+ '1'
                    elif re.match(r'Sum(mer)?',middle):
                        if re.search(r'\d+',j.split()[2],flags=re.M|re.I):
                            j=j.split()[0]+' '+'Jun'+' '+ re.search(r'\d+',j.split()[2],flags=re.M|re.I).group()
                        else:
                            j=j.split()[0]+' '+'Jun'+' '+ '1'
                    elif re.match(r'Spr(ing)?',middle):
                        if re.search(r'\d+',j.split()[2],flags=re.M|re.I):
                            j=j.split()[0]+' '+'Mar'+' '+ re.search(r'\d+',j.split()[2],flags=re.M|re.I).group()
                        else:
                            j=j.split()[0]+' '+'Mar'+' '+ '1'
                    elif re.match(r'Win(ter)?',middle):
                        if re.search(r'\d+',j.split()[2],flags=re.M|re.I):
                            j=j.split()[0]+' '+'Dec'+' '+ re.search(r'\d+',j.split()[2],flags=re.M|re.I).group()
                        else:
                            j=j.split()[0]+' '+'Dec'+' '+ '1'    
                    elif re.match(r'Aut(umn)?',middle) or \
                    re.match(r'Fall',middle):
                        if re.search(r'\d+',j.split()[2],flags=re.M|re.I):
                            j=j.split()[0]+' '+'Sep'+' '+ re.search(r'\d+',j.split()[2],flags=re.M|re.I).group()
                        else:
                            j=j.split()[0]+' '+'Sep'+' '+ '1'      
                    try:
                        date=int(datetime.datetime.strftime(datetime.datetime.strptime(j,'%Y %b %d'), '%Y%m%d'))
                    except:
                        print(j)
                        j=j[:4]
                        date=int(datetime.datetime.strftime(datetime.datetime.strptime(j,'%Y'), '%Y'+'0101'))
                else:
                    j=j[:4]
                    date=int(datetime.datetime.strftime(datetime.datetime.strptime(j,'%Y'), '%Y'+'0101'))
    elif len(j.split())==2:
        if re.match(r'Sum(mer)?',j.split()[1]):
            date=int(j[:4]+'0601')
        elif re.match(r'Spr(ing)?',j.split()[1]):
            date=int(j[:4]+'0301')
        elif re.match(r'Win(ter)?',j.split()[1]):
            date=int(j[:4]+'1201')
        elif re.match(r'Aut(umn)?',j.split()[1]) or re.match(r'Fall',j.split()[1]):
            date=int(j[:4]+'0901')
        elif re.match(r'Sum(mer)?',j.split()[0]):
            date=int(j[-4:]+'0601')
        elif re.match(r'Spr(ing)?',j.split()[0]):
            date=int(j[-4:]+'0301')
        elif re.match(r'Win(ter)?',j.split()[0]):
            date=int(j[-4:]+'1201')
        elif re.match(r'Aut(umn)?',j.split()[0]) or re.match(r'Fall',j.split()[0]):
            date=int(j[-4:]+'0901')
        else:
            try:
                date=int(datetime.datetime.strftime(datetime.datetime.strptime(j,'%Y %b'), '%Y%m'+'01'))
            except:
                if re.search(r'([a-zA-Z]+)[-/]([a-zA-Z]+)',j.split()[1],flags=re.M|re.I) or re.search(r'([a-zA-Z]+)[-/](\d+)',j.split()[1],flags=re.M|re.I):
                    if re.search(r'([a-zA-Z]+)[-/]([a-zA-Z]+)',j.split()[1],flags=re.M|re.I):
                        middle=re.search(r'([a-zA-Z]+)[-/]([a-zA-Z]+)',j.split()[1],flags=re.M|re.I).group(1)
                    else:
                        middle=re.search(r'([a-zA-Z]+)[-/](\d+)',j.split()[1],flags=re.M|re.I).group(1)
                    if middle in months:
                        j=j.split()[0]+' '+middle+' '+ '1'
                    elif re.match(r'Sum(mer)?',middle):
                        j=j.split()[0]+' '+'Jun'+' '+ '1'
                    elif re.match(r'Spr(ing)?',middle):
                        j=j.split()[0]+' '+'Mar'+' '+ '1'
                    elif re.match(r'Win(ter)?',middle):
                        j=j.split()[0]+' '+'Dec'+' '+ '1'    
                    elif re.match(r'Aut(umn)?',middle) or \
                    re.match(r'Fall',middle):
                        j=j.split()[0]+' '+'Sep'+' '+ '1'      
                    try:
                        date=int(datetime.datetime.strftime(datetime.datetime.strptime(j,'%Y %b %d'), '%Y%m%d'))
                    except:
                        j=j[:4]
                        date=int(datetime.datetime.strftime(datetime.datetime.strptime(j,'%Y'), '%Y'+'0101'))
                elif re.search(r'(\d+)[-/](\d+)',j.split()[1],flags=re.M|re.I):
                    midd=re.search(r'(\d+)[-/](\d+)',j.split()[1],flags=re.M|re.I).group(1)
                    if len(midd)!=2:
                        midd='0'+midd
                    j=j[:4]
                    date=int(datetime.datetime.strftime(datetime.datetime.strptime(j,'%Y'), '%Y'+midd+'01'))
                else:
                    print(j)
                    j=j[:4]
                    date=int(datetime.datetime.strftime(datetime.datetime.strptime(j,'%Y'), '%Y'+'0101'))
    elif len(j.split())==1:
        j=j[:4]
        date=int(datetime.datetime.strftime(datetime.datetime.strptime(j,'%Y'), '%Y'+'0101'))
    else:
        j=j.split()[0]+' '+j.split()[1]+' '+re.search(r'\d+',j.split()[2],flags=re.M|re.I).group()
        date=int(datetime.datetime.strftime(datetime.datetime.strptime(j,'%Y %b %d'), '%Y%m%d'))
    i['date']=date
    i['year']=int(str(i.get('date'))[:4])

In [471]:
for i in Gene_list_raw:
    i['addtime']=now_time

In [472]:
Gene_raw=copy.deepcopy(Gene_list_raw)

collections_Gene_raw.remove({'addtime':now_time})

collections_Gene_raw.remove({'addtime':now_time})

In [475]:
if len(Gene_raw)!=0:
    collections_Gene_raw.insert_many(Gene_raw)

collections_Gene_raw.remove({'addtime':now_time})

for i in Gene_raw_list:
    if i.get('Expression'):
        del i['Expression']
    if Gene_converter.get(i.get('entity')).get('Summary'):
        i['Summary']=Gene_converter.get(i.get('entity')).get('Summary')
    else:
        i['Summary']=''
    i['ner']='HALD_Gene'

for i in Gene_raw_list:
    if i.get('addtime')=='2023_02_25':
        print(i)
        break

len(Gene_raw_list)

collections_Gene_raw.drop()

collections_Gene_raw.insert_many(Gene_raw_list)

# Gene_all

In [574]:
Gene_raw_list=list(collections_Gene_raw.find())

In [567]:
for i in Gene_raw_list:
    if i.get('entity')=='CLPS':
        print(i)
        break

{'_id': ObjectId('63fa3d5e648b16758fb9bbca'), 'AB': 'Amongst the main issues challenging the food manufacturing sector, health and nutrition are becoming increasingly important. Global concerns such as obesity, the ageing population and food security will have to be addressed. Food security is not just about assuring food supply, but is also about optimising nutritional delivery from the food that is available [1]. Therefore one challenge is to optimise the health benefits from the lipids and lipid soluble nutrients. Colloid scientists have an affinity for lipids because they are water insoluble, however this presents a challenge to the digestive system, which has to convert them to structures that are less insoluble so they are available for uptake. Despite this, the human digestive system is remarkably effective at digesting and absorbing most lipids. This is primarily driven through maximising energy intake, as lipids possess the highest calorific value, which was a survival trait t

In [568]:
for i in Gene_raw_list:
    i['type']='Gene'
    i['JT']=[i.get('JT')]
    i["PMID"]=[i.get("PMID")]
    i["sentence"]=[[i.get("sentence")]]
    pmid=i.get('PMID')[0]
    i['DP']=[i.get("DP")]
    i['AB']=[i.get("AB")]
    i['URL']='https://pubmed.ncbi.nlm.nih.gov/'+pmid
    i['TI']=[i.get("TI")]
    i['target']=[[i.get('word')]]
    i["date"]=[i.get("date")]
    i["year"]=[i.get("year")]

In [569]:
Gene_mid={}
for i in Gene_raw_list:
    if not Gene_mid.get(i.get("entity")):
        Gene_mid[i.get("entity")]=i
    else:
        if i.get('PMID')[0] in Gene_mid.get(i.get('entity')).get('PMID'):

            ind=Gene_mid.get(i.get('entity')).get('PMID').index(i.get('PMID')[0])
            if i.get("sentence")[0][0] in Gene_mid.get(i.get('entity')).get('sentence')[ind]:
                continue
            else:
                Gene_mid[i.get('entity')]['sentence'][ind] = Gene_mid.get(i.get('entity')).get('sentence')[ind] + i.get("sentence")[0]
                Gene_mid[i.get('entity')]['target'][ind] = Gene_mid.get(i.get('entity')).get('target')[ind] + i.get("target")[0]

        else:
            Gene_mid[i.get("entity")]["PMID"]=Gene_mid.get(i.get("entity")).get("PMID")+i.get("PMID")
            Gene_mid[i.get("entity")]["sentence"]=Gene_mid.get(i.get("entity")).get("sentence")+i.get("sentence")
            Gene_mid[i.get("entity")]["AB"]=Gene_mid.get(i.get("entity")).get("AB")+i.get("AB")
            Gene_mid[i.get("entity")]["DP"]=Gene_mid.get(i.get("entity")).get("DP")+i.get("DP")
            Gene_mid[i.get("entity")]["TI"]=Gene_mid.get(i.get("entity")).get("TI")+i.get("TI")
            Gene_mid[i.get("entity")]["target"]=Gene_mid.get(i.get("entity")).get("target")+i.get("target")
            Gene_mid[i.get("entity")]["date"]=Gene_mid.get(i.get("entity")).get("date")+i.get("date")
            Gene_mid[i.get("entity")]["year"]=Gene_mid.get(i.get("entity")).get("year")+i.get("year")
            Gene_mid[i.get("entity")]["JT"]=Gene_mid.get(i.get("entity")).get("JT")+i.get("JT")

In [570]:
Gene_output_json_final=list(Gene_mid.values())

In [571]:
# 4854
len(Gene_output_json_final)

4854

now_time='2022_12_09'

In [572]:
for i in Gene_output_json_final:
    i['allyears']=[]
    i['datestring']=[]
    for j in i.get('date'):
        if int(str(j)[:4]) not in i.get('allyears'):
            i['allyears']= i.get('allyears')+[int(str(j)[:4])]
    for j in i.get('date'):
        if str(j)[:4]+"-"+str(j)[4:6]+"-"+str(j)[6:] not in i.get('datestring'):
            i['datestring']=i.get('datestring')+[str(j)[:4]+"-"+str(j)[4:6]+"-"+str(j)[6:]]
    i['allyears']=sorted(i.get('allyears'), reverse=True)
    i['datestring']=sorted(i.get('datestring'), reverse=True)
    upperlist=set()
    for j in sum(i.get('target'),[]):
        upperlist.add(j.upper())
    i['targetshapes']=list(upperlist)
    i['addtime']=now_time

In [573]:
Gene_all=copy.deepcopy(Gene_output_json_final)

# RNA_raw

In [240]:
collections_RNA_raw=db.RNA_raw

collections_RNA_raw.remove({'addtime':now_time})

In [241]:
names=locals()

In [ ]:
os.makedirs('../microRNA_lncRNA_rules/'+now_time)

In [ ]:
with open('../microRNA_lncRNA_rules/'+now_time+'/allsentences.txt','a') as f:
    for i in raw_pick_pubtator:
        f.write(i.get('AB')+'\n'+'\n')

In [ ]:
with open('../microRNA_lncRNA_rules/'+now_time+'/run.sh','w') as f:
    f.write('''
#!/bin/bash\n
start_time=$(date +%s)
java -cp "../stanza_corenlp/*" -Xmx500g edu.stanford.nlp.pipeline.StanfordCoreNLP -annotators 'tokenize,ssplit,regexner' -file "../microRNA_lncRNA_rules/'''
+now_time+'''/allsentences.txt" -regexner.mapping ../microRNA_lncRNA_rules/jg-regexner.txt -regexner.ignorecase true -tokenize.options "splitHyphenated=false" --outputFormat json --outputDirectory ../microRNA_lncRNA_rules/'''+now_time+'''/  
end_time=$(date +%s)
cost_time=$[ $end_time-$start_time ]
echo "build kernel time is $(($cost_time/60))min $(($cost_time%60))s"
    
    
    ''')

In [ ]:
import os

In [ ]:
os.system("bash ../microRNA_lncRNA_rules/"+now_time+"/run.sh > ../microRNA_lncRNA_rules/"+now_time+"/run.log")

In [242]:
jsonfile={}
json_file=json.load(open('../microRNA_lncRNA_rules/'+now_time+'/allsentences.txt.json',"r"))
jsonfile["sentences"]=json_file.get("sentences")

In [243]:
import time
import datetime
words=[]
entities_dict_list=[]
starttime=datetime.datetime.now()
l=0
HALDnames=[]
start=0

for HALDname in ["LNCRNA","MICRORNA"]:
    HALDnames.append("HALD_"+HALDname)
    

f=open('../microRNA_lncRNA_rules/'+now_time+'/allsentences.txt')
sentences=f.read()
for i in jsonfile.get("sentences"):
    for HALDname in HALDnames:
        judge=True
        size=len(i.get("tokens"))
        for k,j in enumerate(i.get("tokens")):
            if j.get("ner"):
                if j.get("ner")==HALDname:
                    combination=j.get('word')
                    words.append(combination)
                    entities_dict={}
                    entities_dict["word"]=combination
                    entities_dict["wordBegin"]=j.get("characterOffsetBegin")+start
                    entities_dict["wordEnd"]=j.get("characterOffsetEnd")+start
                    entities_dict["sentenceBegin"]=i.get("tokens")[0].get("characterOffsetBegin")+start
                    entities_dict["sentenceEnd"]=i.get("tokens")[-1].get("characterOffsetEnd")+start
                    entities_dict["sentence"]=sentences[i.get("tokens")[0].get("characterOffsetBegin"):i.get("tokens")[-1].get("characterOffsetEnd")]
                    entities_dict["ner"]=HALDname
                    entities_dict_list.append(entities_dict)


                    if judge:
                        names['l'+HALDname]=k+1
                        judge=False
                    if names['l'+HALDname]<k:
                        names['l'+HALDname]=k+1

                    if k != names['l'+HALDname] -1:
                        continue


                    if names['l'+HALDname]<size:
                        if i.get("tokens")[names['l'+HALDname]].get("ner"):
                            if i.get("tokens")[names['l'+HALDname]].get("ner")==HALDname:
                                while i.get("tokens")[names['l'+HALDname]].get("ner")==HALDname:
                                    combination+=" "+i.get("tokens")[names['l'+HALDname]].get("word")
                                    names['l'+HALDname]+=1
                                    if names['l'+HALDname]>=size:
                                        break
                                names['l'+HALDname]+=1
                                words.append(combination)
                                entities_dict={}
                                entities_dict["word"]=combination
                                entities_dict["wordBegin"]=j.get("characterOffsetBegin")+start
                                entities_dict["wordEnd"]=j.get("characterOffsetEnd")+start
                                entities_dict["sentenceBegin"]=i.get("tokens")[0].get("characterOffsetBegin")+start
                                entities_dict["sentenceEnd"]=i.get("tokens")[-1].get("characterOffsetEnd")+start
                                entities_dict["sentence"]=sentences[i.get("tokens")[0].get("characterOffsetBegin"):i.get("tokens")[-1].get("characterOffsetEnd")]
                                entities_dict["ner"]=HALDname
                                entities_dict_list.append(entities_dict)

f.close()
start+=len(sentences)
                    
endtime=datetime.datetime.now()
print("The running time is "+str((endtime-starttime).seconds)+"s")

The running time is 0s


In [244]:
len(entities_dict_list)

124

In [245]:
entities_dict_list[2]

{'word': 'MiR-942-5p',
 'wordBegin': 75004,
 'wordEnd': 75014,
 'sentenceBegin': 75004,
 'sentenceEnd': 75106,
 'sentence': 'MiR-942-5p downregulation reversed si-circ_0004104-mediated influences in HUVECs upon ox-LDL exposure.',
 'ner': 'HALD_MICRORNA'}

In [246]:
length2pmid=[]
start=0
for i in raw_pick_pubtator:
    mid=[]
    mid.append(start)
    mid.append(i.get('PMID'))
    length2pmid.append(mid)
    start+=len(i.get('AB'))+2

In [247]:
len(length2pmid)

4398

In [248]:
length2pmid[10]

[18177, '36466890']

In [249]:
k=0
for i in entities_dict_list:
    for j in range(k,len(length2pmid)):
        if i.get('wordBegin')<length2pmid[j][0]:
            i['PMID']=length2pmid[j-1][1]
            i['AB']=pubtator_raw_dict2AB.get(i.get('PMID'))
            k=j-1
            break
    else:
        i['PMID']=length2pmid[-1][1]
        i['AB']=pubtator_raw_dict2AB.get(i.get('PMID'))

In [250]:
entities_dict_list[10]

{'word': 'miR-34a-5p',
 'wordBegin': 463865,
 'wordEnd': 463875,
 'sentenceBegin': 463845,
 'sentenceEnd': 463934,
 'sentence': 'The binding between miR-34a-5p and SIRT1 was confirmed by dual-luciferase reporter assay.',
 'ner': 'HALD_MICRORNA',
 'PMID': '36401446',
 'AB': 'Chronic obstructive pulmonary disease (COPD) is characterized by accelerated lung aging. Smoking is the critical risk factor for COPD. Cellular senescence of airway epithelial cells is the cytological basis of accelerated lung aging in COPD, and the regulation of microRNAs (miRNAs) is the central epigenetic mechanism of cellular senescence. Resveratrol (Res) is a polyphenol with anti-aging properties. This study investigated whether Res attenuates cigarette smoke extract (CSE)-induced cellular senescence in human airway epithelial cells (BEAS-2B) through the miR-34a/SIRT1/nuclear factor-kappaB (NF-kappaB) pathway. BEAS-2B cells were treated with Res, CSE and transfected with miR-34a-5p mimics. Cellular senescence wa

In [254]:
import pandas as pd

In [255]:
RNA2ncbi=pd.read_csv('../ncbi/RNA_ncbi.txt',sep='\t',encoding='utf-8',converters = {'NCBI Gene ID(supplied by NCBI)':str})

In [256]:
RNA2approved_dict=dict(zip(RNA2ncbi['NCBI Gene ID(supplied by NCBI)'],RNA2ncbi['Approved symbol'] ))

In [257]:
RNA2ncbi_dict=dict(zip(RNA2ncbi['Approved symbol'], RNA2ncbi['NCBI Gene ID(supplied by NCBI)']))

In [263]:
# add entity
error_RNA=[]
pubtator_RNAs=[]
for i in pubtator_gene:
    if RNA2approved_dict.get(i.get('identifier')):
        i['entity']=RNA2approved_dict.get(i.get('identifier'))
        pubtator_RNAs.append(i)
    else:
        error_RNA.append(i)

In [264]:
len(error_RNA)

7165

In [265]:
len(pubtator_RNAs)

81

In [266]:
names=locals()
HALDnames=[]
for HALDname in ["CLUSTERRNA","LNCRNA","MICRORNA","MISCRNA","RIBOSOMALRNA","SMALL_NUCLEARRNA","SMALL_NUCLEOLARRNA","TRANSFERRNA","VAULTRNA","YRNA"]:
    HALDnames.append("HALD_"+HALDname)

for name in ["clusterRNA","lncRNA","microRNA","miscRNA","ribosomalRNA","small_nuclearRNA","small_nucleolarRNA","transferRNA","vaultRNA","yRNA"]:
    file="../dict/"+name+"/"+name+"dict.json"
    with open(file,"r") as f:
        jsons=json.load(f)
        json_lower={}
        for i,j in jsons.items():
            json_lower[i.lower()]=j
        names[name+"_dict"]=json_lower

In [267]:
pattern=['([Hh][Ss][Aa])?[\-]?[Mm][Ii][Cc][Rr][Oo][Rr][Nn][Aa][\-]?\d+[a-zA-Z]?[\-]?[12345]?[\-]?[35]?[Pp]?',
'([Hh][Ss][Aa])?[\-]?[Mm][Ii][Cc][Rr][Oo][Rr][Nn][Aa] \d+[a-zA-Z]?[\-]?[12345]?[\-]?[35]?[Pp]?',
'([Hh][Ss][Aa])?[\-]?[Mm][Ii][Rr][Nn]?[\-]?\d+[a-zA-Z]?[\-]?[12345]?[\-]?[35]?[Pp]?',
'([Hh][Ss][Aa])?[\-]?[Mm][Ii][Rr][Nn]? \d+[a-zA-Z]?[\-]?[12345]?[\-]?[35]?[Pp]?',
'([Hh][Ss][Aa])?[\-]?([Mm][Ii][Cc][Rr][Oo][Rr][Nn][Aa][\-]?\d+)([a-z][/]?)+',
'([Hh][Ss][Aa])?[\-]?[Mm][Ii][Cc][Rr][Oo][Rr][Nn][Aa] \d+([a-z][/]?)+',
'([Hh][Ss][Aa])?[\-]?([Mm][Ii][Rr][Nn]?[\-]?\d+)([a-z][/]?)+',
'([Hh][Ss][Aa])?[\-]?[Mm][Ii][Rr][Nn]? \d+([a-z][/]?)+',
'[Hh][Ss][Aa][\-][Mm][Ii][Rr][Nn]?[\-]\d+[a-zA-Z]?[\-]?[12345]?[\-]?[35]?[Pp]?',
'([Hh][Ss][Aa][\-][Mm][Ii][Rr][Nn]?[\-]\d+)([a-z][/]?)+',
'[Mm][Ii][Rr][Nn]?[Ll][Ee][Tt]7[A-Ia-i][123]?',
'([Mm][Ii][Rr][Nn]?[Ll][Ee][Tt]7)([A-Ia-i][/]?)+',
'[Mm][Ii][Cc][Rr][Oo][Rr][Nn][Aa][\-][Ll][Ee][Tt][\-]\d+[a-zA-Z]?[\-]?[12345]?[\-]?[35]?[Pp]?',
'[Mm][Ii][Cc][Rr][Oo][Rr][Nn][Aa] [Ll][Ee][Tt] \d+[a-zA-Z]?[\-]?[12345]?[\-]?[35]?[Pp]?',
'[Mm][Ii][Rr][Nn]?[\-][Ll][Ee][Tt][\-]\d+[a-zA-Z]?[\-]?[12345]?[\-]?[35]?[Pp]?',
'[Mm][Ii][Rr][Nn]? [Ll][Ee][Tt] \d+[a-zA-Z]?[\-]?[12345]?[\-]?[35]?[Pp]?',
'([Mm][Ii][Cc][Rr][Oo][Rr][Nn][Aa][\-][Ll][Ee][Tt][\-]\d+)([a-z][/]?)+',
'[Mm][Ii][Cc][Rr][Oo][Rr][Nn][Aa] [Ll][Ee][Tt] \d+([a-z][/]?)+',
'([Mm][Ii][Rr][Nn]?[\-][Ll][Ee][Tt][\-]\d+)([a-z][/]?)+',
'[Mm][Ii][Rr][Nn]? [Ll][Ee][Tt] \d+([a-z][/]?)+']

In [268]:
# add RNA type
filtered_pubtator_RNAs=[]
drop_RNAs=[]
import re
midcount1=0
midcount2=0
for i in pubtator_RNAs:
    for name in ["clusterRNA","lncRNA","microRNA","miscRNA","ribosomalRNA","small_nuclearRNA","small_nucleolarRNA","transferRNA","vaultRNA","yRNA"]:
        if names[name+"_dict"].get(i.get('text').lower()):
            i['ner']="HALD_"+name.upper()
            i['type']="RNA"
            i['specific_type']=name.upper()
            filtered_pubtator_RNAs.append(i)
            break
    else:
        for j in pattern:
            if re.search(j,i.get('text').lower()):
                midcount1+=1
                i['ner']="HALD_MICRORNA"
                i['type']="RNA"
                i['specific_type']="MICRORNA"
                filtered_pubtator_RNAs.append(i)
                break
            elif re.search(r'linc\d{5}',i.get('text').lower()):
                midcount2+=1
                i['ner']="HALD_LINCRNA"
                i['type']="RNA"
                i['specific_type']="LINCRNA"
                filtered_pubtator_RNAs.append(i)
                break
        else:
            drop_RNAs.append(i)

In [269]:
print(midcount1)
print(midcount2)

10
0


In [270]:
len(drop_RNAs)

7

In [271]:
filtered_pubtator_genes_pmid2word={}
for i in filtered_pubtator_RNAs:
    filtered_pubtator_genes_pmid2word[i.get('id')]=filtered_pubtator_genes_pmid2word.get(i.get('id'),[])+[i.get('text')]

In [272]:
len(filtered_pubtator_genes_pmid2word)

18

In [273]:
# based on dict. Whether in pubtator
RNA_entities_list=[]
for i in entities_dict_list:
    if filtered_pubtator_genes_pmid2word.get(i.get('PMID')):
        if i.get('word') not in filtered_pubtator_genes_pmid2word.get(i.get('PMID')):
            RNA_entities_list.append(i)
    else:
        RNA_entities_list.append(i)

In [274]:
len(RNA_entities_list)

71

In [275]:
len(entities_dict_list)

124

In [276]:
RNA_entities_list[11]

{'word': 'miR-34a-5p',
 'wordBegin': 464531,
 'wordEnd': 464541,
 'sentenceBegin': 464446,
 'sentenceEnd': 464560,
 'sentence': 'Moreover, Res reversed the CSE-induced down-regulation of SIRT1 and up-regulation of miR-34a-5p and NF-kappaB p65.',
 'ner': 'HALD_MICRORNA',
 'PMID': '36401446',
 'AB': 'Chronic obstructive pulmonary disease (COPD) is characterized by accelerated lung aging. Smoking is the critical risk factor for COPD. Cellular senescence of airway epithelial cells is the cytological basis of accelerated lung aging in COPD, and the regulation of microRNAs (miRNAs) is the central epigenetic mechanism of cellular senescence. Resveratrol (Res) is a polyphenol with anti-aging properties. This study investigated whether Res attenuates cigarette smoke extract (CSE)-induced cellular senescence in human airway epithelial cells (BEAS-2B) through the miR-34a/SIRT1/nuclear factor-kappaB (NF-kappaB) pathway. BEAS-2B cells were treated with Res, CSE and transfected with miR-34a-5p mimic

In [277]:
import re
for i in RNA_entities_list:
    i['type']='RNA'
    i['specific_type']=i.get('ner')[5:]

In [278]:
RNAdict=json.load(open('../dict/RNAdict.json','r'))

In [279]:
# 考虑了大小写引起的可能不是一个实体的问题
# 验证后发现不会发生
RNAdict_lower={}
for i,j in RNAdict.items():
    RNAdict_lower[i.lower()]=RNAdict_lower.get(i.lower(),[])+j
    RNAdict_lower[i.lower()]=list(set(RNAdict_lower.get(i.lower())))

In [280]:
cccc=0
final_RNA_entities_list=[]
drop_RNA_entities_list=[]
for i in RNA_entities_list:
    if RNAdict_lower.get(i.get('word').lower()):
        for j in RNAdict_lower.get(i.get('word').lower()):
            i['entity']=j
            final_RNA_entities_list.append(i)
    elif re.search(r'-[3,5]p',i.get('word')):
            if RNAdict_lower.get(re.sub(r'-[3,5]p','',i.get('word'))):
                for j in RNAdict_lower.get(re.sub(r'-[3,5]p','',i.get('word'))):
                    i['entity']=j
                    final_RNA_entities_list.append(i)
                    cccc+=1
    else:
        drop_RNA_entities_list.append(i)

In [281]:
len(final_RNA_entities_list)

14

In [282]:
len(drop_RNA_entities_list)

6

In [283]:
drop_RNA_entities_list[0]

{'word': 'MiR-181a',
 'wordBegin': 2133730,
 'wordEnd': 2133738,
 'sentenceBegin': 2133721,
 'sentenceEnd': 2133844,
 'sentence': 'METHODS: MiR-181a was isolated from plasma according to standardized procedures and its expression levels measured by qPCR.',
 'ner': 'HALD_MICRORNA',
 'PMID': '36175844',
 'AB': 'BACKGROUND: Chronic low-level inflammation is thought to play a role in many age-related diseases and to contribute to multimorbidity and to the disability related to this condition. In this framework, inflamma-miRs, an important subset of miRNA able to regulate inflammation molecules, appear to be key players. This study aimed to evaluate plasma levels of the inflamma-miR-181a in relation to age, parameters of health status (clinical, physical, and cognitive) and indices of multimorbidity in a cohort of 244 subjects aged 65- 97. METHODS: MiR-181a was isolated from plasma according to standardized procedures and its expression levels measured by qPCR. Correlation tests and multiva

In [284]:
for i in filtered_pubtator_RNAs:
    i['PMID']=i.get('id')
    i['word']=i.get('text')

In [285]:
final_RNA=filtered_pubtator_RNAs+final_RNA_entities_list

In [286]:
len(final_RNA)

88

In [287]:
RNAs_inte_copy=copy.deepcopy(final_RNA)

In [291]:
deduplicated_RNA=set()
deduplicated_RNA_count=0
RNA_list_raw=[]
for i in RNAs_inte_copy:
    k=i.get('PMID')
    i['DP']=rawpickwithAB.get(k).get('DP')
    i['AB']=rawpickwithAB.get(k).get('AB')
    i['JT']=rawpickwithAB.get(k).get('JT')
    i['URL']='https://pubmed.ncbi.nlm.nih.gov/'+k
    i['TI']=rawpickwithAB.get(k).get('TI')
    i['target']=i.get('word')
    tuples=(i.get("PMID"),i.get("entity"),i.get("word"),i.get("sentence"))
    if tuples not in deduplicated_RNA:
        deduplicated_RNA.add(tuples)
        RNA_list_raw.append(i)
    else:
        deduplicated_RNA_count+=1
        continue

In [292]:
len(RNA_list_raw)

82

In [293]:
RNA2blank={}
for i in RNA_list_raw:
    RNA2blank[i.get('entity')]=''

In [294]:
len(RNA2blank)

22

In [295]:
RNA_converter=pickle.load(open('../ncbi/RNA_converter.pkl','rb'))

In [296]:
len(RNA_converter)

339

In [297]:
import time
import datetime
from multiprocessing import Process, Manager, Pool
import os, time, random
from lxml import etree
import requests
def get_RNA_info(i,final_RNAs_inte):
    ncbiid=RNA2ncbi_dict.get(i)
    ncbiurl='https://www.ncbi.nlm.nih.gov/gene/' + ncbiid
    headers = {
        "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 \
        (KHTML, like Gecko) Chrome/83.0.4103.97 Safari/537.36"
    }
    url = 'https://www.ncbi.nlm.nih.gov/gene/'+ncbiid
    
    res=requests.get(url=url,headers=headers)
    html = etree.HTML(res.content)
    ncbisummary={
    # 'numbersOfLiterature' : len(i.get('PMID')),
    'Official_Symbol' : html.xpath('//*[@id="summaryDl"]/dt[contains(.//text(),"Official")]/following::dd[1]/text()')[0],
    'Official_Full_Name' : html.xpath('//*[@id="summaryDl"]/dt[contains(.//text(),"Official")]/following::dd[1]/text()')[1],
    # 'Also_known_as' : ''.join(html.xpath('//*[@id="summaryDl"]/dt[contains(.//text(),"Also known as")]/following::dd[1]/text()')).replace('HGNC:HGNC:',''),
    'HGNC_ID' : ''.join(html.xpath('//*[@id="summaryDl"]/dt[contains(.//text(),"Primary source")]/following::dd[1]/a/text()')),
    'HGNC_URL' : ''.join(html.xpath('//*[@id="summaryDl"]/dt[contains(.//text(),"Primary source")]/following::dd[1]/a/@href')),
    'Ensembl_ID' :  ''.join(html.xpath('//*[@id="summaryDl"]/dt[contains(.//text(),"See related")]/following::dd[1]/a[1]/text()')),
    'Ensembl_URL' : ''.join(html.xpath('//*[@id="summaryDl"]/dt[contains(.//text(),"See related")]/following::dd[1]/a[1]/@href')),
    'MIM_ID' : ''.join(html.xpath('//*[@id="summaryDl"]/dt[contains(.//text(),"See related")]/following::dd[1]/a[2]/text()')),
    'MIM_URL' : ''.join(html.xpath('//*[@id="summaryDl"]/dt[contains(.//text(),"See related")]/following::dd[1]/a[2]/@href')),
    'AllianceGenome_ID' : ''.join(html.xpath('//*[@id="summaryDl"]/dt[contains(.//text(),"See related")]/following::dd[1]/a[3]/text()')),
    'AllianceGenome_URL' : ''.join(html.xpath('//*[@id="summaryDl"]/dt[contains(.//text(),"See related")]/following::dd[1]/a[3]/@href')),
    'NCBI_ID' : ncbiid ,
    'NCBI_URL' : ncbiurl
    }
    try:
        also=''.join(html.xpath('//*[@id="summaryDl"]/dt[contains(.//text(),"Also known as")]/following::dd[1]/text()')).replace('HGNC:HGNC:','')
    except:
        also=''
    finally:
        ncbisummary.update({'Also_known_as' : also})
    try:
        also2=''.join(html.xpath('//*[@id="summaryDl"]/dt[contains(.//text(),"Summary")]/following::dd[1]/text()'))
    except:
        also2=''
    finally:
        ncbisummary.update({'Summary' : also2})
    final_RNAs_inte[i]=ncbisummary
    # print(ncbisummary)
    time.sleep(0.5)

In [298]:
judge=False
p = Pool(10)
manager=Manager()
final_RNAs_inte = manager.dict()
for i in RNA2blank.keys():
    if not RNA_converter.get(i):
        p.apply_async(get_RNA_info, args=(i,final_RNAs_inte))
        judge=True
p.close()
p.join()
if judge:
    RNA_converter.update(dict(final_RNAs_inte))
    pickle.dump(RNA_converter, open('../ncbi/RNA_converter.pkl','wb'))

In [299]:
len(RNA_converter)

339

In [300]:
for i in range(len(RNA_list_raw)):
    RNA_list_raw[i]=dict(RNA_list_raw[i],**RNA_converter.get(RNA_list_raw[i].get('entity')))

In [301]:
import time
import datetime
import re
dropout=[]
months=['Jan','Feb','Mar','Apr','May','Jun','Jul','Aug','Sep','Oct','Nov','Dec']
import re

for i in RNA_list_raw:
    j=i.get('DP')
    if re.search('[S,s]upplement',j,flags=re.M|re.I):
        j=re.sub('[S,s]upplement.*','',j,flags=re.M|re.I)
    j=re.sub('\s?-\s?','-',j)
    j=re.sub('\.','',j)
    j=re.sub('Aout','Aug',j)
    if len(j.split(' '))==3:
        try:
            date=int(datetime.datetime.strftime(datetime.datetime.strptime(j,'%Y %b %d'), '%Y%m%d'))
        except:
            if j.split()[1] in months:
                if re.search(r'\d+',j.split()[2],flags=re.M|re.I):
                    j=j.split()[0]+' '+j.split()[1]+' '+ re.search(r'\d+',j.split()[2],flags=re.M|re.I).group()
                    try:
                        date=int(datetime.datetime.strftime(datetime.datetime.strptime(j,'%Y %b %d'), '%Y%m%d'))
                    except:
                        j=j[:4]
                        date=int(datetime.datetime.strftime(datetime.datetime.strptime(j,'%Y'), '%Y'+'0101'))
                        print('***************************************ERROR**********************************************')
                else:
                    j=j.split()[0]+' '+j.split()[1]+' '+ '1'
                    try:
                        date=int(datetime.datetime.strftime(datetime.datetime.strptime(j,'%Y %b %d'), '%Y%m%d'))
                    except:
                        print(j)
                        j=j[:4]
                        date=int(datetime.datetime.strftime(datetime.datetime.strptime(j,'%Y'), '%Y'+'0101'))
            elif j.split()[1] not in months:
                if re.search(r'([a-zA-Z]+)[-/]([a-zA-Z]+)',j.split()[1],flags=re.M|re.I) or re.search(r'([a-zA-Z]+)[-/](\d+)',j.split()[1],flags=re.M|re.I):
                    if re.search(r'([a-zA-Z]+)[-/]([a-zA-Z]+)',j.split()[1],flags=re.M|re.I):
                        middle=re.search(r'([a-zA-Z]+)[-/]([a-zA-Z]+)',j.split()[1],flags=re.M|re.I).group(1)
                    else:
                        middle=re.search(r'([a-zA-Z]+)[-/](\d+)',j.split()[1],flags=re.M|re.I).group(1)
                    if middle in months:
                        if re.search(r'\d+',j.split()[2],flags=re.M|re.I):
                            j=j.split()[0]+' '+middle+' '+ re.search(r'\d+',j.split()[2],flags=re.M|re.I).group()
                        else:
                            j=j.split()[0]+' '+middle+' '+ '1'
                    elif re.match(r'Sum(mer)?',middle):
                        if re.search(r'\d+',j.split()[2],flags=re.M|re.I):
                            j=j.split()[0]+' '+'Jun'+' '+ re.search(r'\d+',j.split()[2],flags=re.M|re.I).group()
                        else:
                            j=j.split()[0]+' '+'Jun'+' '+ '1'
                    elif re.match(r'Spr(ing)?',middle):
                        if re.search(r'\d+',j.split()[2],flags=re.M|re.I):
                            j=j.split()[0]+' '+'Mar'+' '+ re.search(r'\d+',j.split()[2],flags=re.M|re.I).group()
                        else:
                            j=j.split()[0]+' '+'Mar'+' '+ '1'
                    elif re.match(r'Win(ter)?',middle):
                        if re.search(r'\d+',j.split()[2],flags=re.M|re.I):
                            j=j.split()[0]+' '+'Dec'+' '+ re.search(r'\d+',j.split()[2],flags=re.M|re.I).group()
                        else:
                            j=j.split()[0]+' '+'Dec'+' '+ '1'    
                    elif re.match(r'Aut(umn)?',middle) or \
                    re.match(r'Fall',middle):
                        if re.search(r'\d+',j.split()[2],flags=re.M|re.I):
                            j=j.split()[0]+' '+'Sep'+' '+ re.search(r'\d+',j.split()[2],flags=re.M|re.I).group()
                        else:
                            j=j.split()[0]+' '+'Sep'+' '+ '1'      
                    try:
                        date=int(datetime.datetime.strftime(datetime.datetime.strptime(j,'%Y %b %d'), '%Y%m%d'))
                    except:
                        print(j)
                        j=j[:4]
                        date=int(datetime.datetime.strftime(datetime.datetime.strptime(j,'%Y'), '%Y'+'0101'))
                else:
                    j=j[:4]
                    date=int(datetime.datetime.strftime(datetime.datetime.strptime(j,'%Y'), '%Y'+'0101'))
    elif len(j.split())==2:
        if re.match(r'Sum(mer)?',j.split()[1]):
            date=int(j[:4]+'0601')
        elif re.match(r'Spr(ing)?',j.split()[1]):
            date=int(j[:4]+'0301')
        elif re.match(r'Win(ter)?',j.split()[1]):
            date=int(j[:4]+'1201')
        elif re.match(r'Aut(umn)?',j.split()[1]) or re.match(r'Fall',j.split()[1]):
            date=int(j[:4]+'0901')
        elif re.match(r'Sum(mer)?',j.split()[0]):
            date=int(j[-4:]+'0601')
        elif re.match(r'Spr(ing)?',j.split()[0]):
            date=int(j[-4:]+'0301')
        elif re.match(r'Win(ter)?',j.split()[0]):
            date=int(j[-4:]+'1201')
        elif re.match(r'Aut(umn)?',j.split()[0]) or re.match(r'Fall',j.split()[0]):
            date=int(j[-4:]+'0901')
        else:
            try:
                date=int(datetime.datetime.strftime(datetime.datetime.strptime(j,'%Y %b'), '%Y%m'+'01'))
            except:
                if re.search(r'([a-zA-Z]+)[-/]([a-zA-Z]+)',j.split()[1],flags=re.M|re.I) or re.search(r'([a-zA-Z]+)[-/](\d+)',j.split()[1],flags=re.M|re.I):
                    if re.search(r'([a-zA-Z]+)[-/]([a-zA-Z]+)',j.split()[1],flags=re.M|re.I):
                        middle=re.search(r'([a-zA-Z]+)[-/]([a-zA-Z]+)',j.split()[1],flags=re.M|re.I).group(1)
                    else:
                        middle=re.search(r'([a-zA-Z]+)[-/](\d+)',j.split()[1],flags=re.M|re.I).group(1)
                    if middle in months:
                        j=j.split()[0]+' '+middle+' '+ '1'
                    elif re.match(r'Sum(mer)?',middle):
                        j=j.split()[0]+' '+'Jun'+' '+ '1'
                    elif re.match(r'Spr(ing)?',middle):
                        j=j.split()[0]+' '+'Mar'+' '+ '1'
                    elif re.match(r'Win(ter)?',middle):
                        j=j.split()[0]+' '+'Dec'+' '+ '1'    
                    elif re.match(r'Aut(umn)?',middle) or \
                    re.match(r'Fall',middle):
                        j=j.split()[0]+' '+'Sep'+' '+ '1'      
                    try:
                        date=int(datetime.datetime.strftime(datetime.datetime.strptime(j,'%Y %b %d'), '%Y%m%d'))
                    except:
                        j=j[:4]
                        date=int(datetime.datetime.strftime(datetime.datetime.strptime(j,'%Y'), '%Y'+'0101'))
                elif re.search(r'(\d+)[-/](\d+)',j.split()[1],flags=re.M|re.I):
                    midd=re.search(r'(\d+)[-/](\d+)',j.split()[1],flags=re.M|re.I).group(1)
                    if len(midd)!=2:
                        midd='0'+midd
                    j=j[:4]
                    date=int(datetime.datetime.strftime(datetime.datetime.strptime(j,'%Y'), '%Y'+midd+'01'))
                else:
                    print(j)
                    j=j[:4]
                    date=int(datetime.datetime.strftime(datetime.datetime.strptime(j,'%Y'), '%Y'+'0101'))
    elif len(j.split())==1:
        j=j[:4]
        date=int(datetime.datetime.strftime(datetime.datetime.strptime(j,'%Y'), '%Y'+'0101'))
    else:
        j=j.split()[0]+' '+j.split()[1]+' '+re.search(r'\d+',j.split()[2],flags=re.M|re.I).group()
        date=int(datetime.datetime.strftime(datetime.datetime.strptime(j,'%Y %b %d'), '%Y%m%d'))
    i['date']=date
    i['year']=int(str(i.get('date'))[:4])

In [302]:
RNA_list_raw[0]

{'id': '36460954',
 'AB': 'BACKGROUND: Cardiovascular disease was the most common disease among the elderly with high morbidity and mortality. Circ_0004104 was demonstrated to be involved in the regulation of atherosclerosis. METHODS: Quantitative real-time polymerase chain reaction was employed to measure the expression of circ_0004104, miR-942-5p and Rho associated coiled-coil containing protein kinase 2 (ROCK2). Cell proliferation was tested by 3-(4,5-dimethyl-2-thiazolyl)-2,5-diphenyl-2-H-tetrazolium bromide (MTT) assay. Cell apoptosis was measured by flow cytometry, and tube formation assay was used to detect the angiogenesis ability of cells. Western blot assay was performed to assess protein levels. Enzyme-linked immunosorbent assay was used to detect the release of IL-1beta and TNF-alpha. The relationship between miR-942-5p and circ_0004104 or ROCK2 was identified by dual-luciferase reporter assay, RNA immunoprecipitation (RIP) assay, and RNA pull-down assay. RESULTS: Oxidized 

In [303]:
for i in RNA_list_raw:
    i['addtime']=now_time

In [322]:
for i,j in RNA_converter.items():
    if j.get('Summary') and j.get('Summary')[0]=='\n':
        print(i)

collections_RNA_raw.remove({'addtime':now_time})

In [305]:
if len(RNA_list_raw)!=0:
    collections_RNA_raw.insert_many(RNA_list_raw)

collections_RNA_raw.remove({'addtime':now_time})

In [307]:
for i in RNA_raw_list:
    if RNA_converter.get(i.get('entity')).get('Summary'):
        i['Summary']=RNA_converter.get(i.get('entity')).get('Summary')
    else:
        i['Summary']=''

In [325]:
for j in RNA_all:
    if j.get('Summary') and j.get('Summary')[0]=='\n':
        print(i)

In [308]:
len(RNA_raw_list)

2492

In [309]:
collections_RNA_raw.drop()

In [310]:
collections_RNA_raw.insert_many(RNA_raw_list)

# RNA_all

In [421]:
collections_RNA_all=db.RNA_all

In [438]:
RNA_raw_list=list(collections_RNA_raw.find())

In [439]:
for i in RNA_raw_list:
    i['type']='RNA'
    i["PMID"]=[i.get("PMID")]
    i["sentence"]=[[i.get("sentence")]]
    pmid=i.get('PMID')[0]
    i['DP']=[i.get("DP")]
    i['AB']=[i.get("AB")]
    i['URL']='https://pubmed.ncbi.nlm.nih.gov/'+pmid
    i['TI']=[i.get("TI")]
    i['target']=[[i.get('word')]]
    i["date"]=[i.get("date")]
    i["year"]=[i.get("year")]
    i['JT']=[i.get('JT')]

In [440]:
RNA_mid={}
for i in RNA_raw_list:
    if not RNA_mid.get(i.get("entity")):
        RNA_mid[i.get("entity")]=i
    else:
        if i.get('PMID')[0] in RNA_mid.get(i.get('entity')).get('PMID'):

            ind=RNA_mid.get(i.get('entity')).get('PMID').index(i.get('PMID')[0])
            if i.get("sentence")[0][0] in RNA_mid.get(i.get('entity')).get('sentence')[ind]:
                continue
            else:
                RNA_mid[i.get('entity')]['sentence'][ind] = RNA_mid.get(i.get('entity')).get('sentence')[ind] + i.get("sentence")[0]
                RNA_mid[i.get('entity')]['target'][ind] = RNA_mid.get(i.get('entity')).get('target')[ind] + i.get("target")[0]

        else:
            RNA_mid[i.get("entity")]["PMID"]=RNA_mid.get(i.get("entity")).get("PMID")+i.get("PMID")
            RNA_mid[i.get("entity")]["sentence"]=RNA_mid.get(i.get("entity")).get("sentence")+i.get("sentence")
            RNA_mid[i.get("entity")]["AB"]=RNA_mid.get(i.get("entity")).get("AB")+i.get("AB")
            RNA_mid[i.get("entity")]["DP"]=RNA_mid.get(i.get("entity")).get("DP")+i.get("DP")
            RNA_mid[i.get("entity")]["TI"]=RNA_mid.get(i.get("entity")).get("TI")+i.get("TI")
            RNA_mid[i.get("entity")]["target"]=RNA_mid.get(i.get("entity")).get("target")+i.get("target")
            RNA_mid[i.get("entity")]["date"]=RNA_mid.get(i.get("entity")).get("date")+i.get("date")
            RNA_mid[i.get("entity")]["year"]=RNA_mid.get(i.get("entity")).get("year")+i.get("year")
            RNA_mid[i.get("entity")]["JT"]=RNA_mid.get(i.get("entity")).get("JT")+i.get("JT")

In [441]:
RNA_output_json_final=list(RNA_mid.values())

In [442]:
len(RNA_output_json_final)

339

In [443]:
for i in RNA_output_json_final:
    i['allyears']=[]
    i['datestring']=[]
    for j in i.get('date'):
        if int(str(j)[:4]) not in i.get('allyears'):
            i['allyears']= i.get('allyears')+[int(str(j)[:4])]
    for j in i.get('date'):
        if str(j)[:4]+"-"+str(j)[4:6]+"-"+str(j)[6:] not in i.get('datestring'):
            i['datestring']=i.get('datestring')+[str(j)[:4]+"-"+str(j)[4:6]+"-"+str(j)[6:]]
    i['allyears']=sorted(i.get('allyears'), reverse=True)
    i['datestring']=sorted(i.get('datestring'), reverse=True)
    upperlist=set()
    for j in sum(i.get('target'),[]):
        upperlist.add(j.upper())
    i['targetshapes']=list(upperlist)
    i['addtime']=now_time

collections_RNA_all.drop()

In [444]:
RNA_all=copy.deepcopy(RNA_output_json_final)

In [168]:
# collections_RNA_all.insert_many(RNA_all)

In [169]:
# json.dump(RNA_all_copy,open('results/RNA_all.json','w'))

# chemical_raw

In [170]:
chemical_pubtator_addpmid={}
for i in pubtator_raw:
    if i.get('type')=='Chemical':
        if chemical_pubtator_addpmid.get(i.get('id')):
            chemical_pubtator_addpmid[i.get('id')]=chemical_pubtator_addpmid.get(i.get('id'))+[i]
        else:
            chemical_pubtator_addpmid[i.get('id')]=[i]

In [171]:
print(len(chemical_pubtator_addpmid))

1592


In [172]:
chemical_output_jsons=list(chemical_pubtator_addpmid.values())

In [173]:
chemical_output_json=[]
for i in chemical_output_jsons:
    for j in i:
        chemical_output_json.append(j)

In [174]:
chemical_raw=copy.deepcopy(chemical_output_json)

In [175]:
pwd

'/root/hald_autoupdate/2022_12_09'

In [176]:
pairs=['Toxin','Peptide','Lipid','Protein','Carbohydrate','Pharmaceutical Preparations']

In [177]:
names=locals()

In [178]:
alllist=[]
for j in pairs:
    names[j.replace(' ','_').lower()]=pickle.load(open('../chemical/'+j.replace(' ','_')+'/'+j.replace(' ','_')+'.pkl','rb'))
    alllist+=names[j.replace(' ','_').lower()][1:]

In [719]:
alllist[0]

{'text': 'Bacterial Toxins',
 'url': '/mesh/68001427',
 'MeSHUniqueID': 'D001427',
 'type': 'Toxin'}

In [179]:
for i in alllist:
    if not i.get('MeSHUniqueID'):
        i['MeSHUniqueID']='D'+i.get('url')[-6:]

In [180]:
chemicallist=[]
MESHlist=[]
notchemicallist=[]
MESHurl=[]
for i in alllist:
    if i.get('MeSHUniqueID'):
        chemicallist.append({'MESH':'MESH:'+i.get('MeSHUniqueID'),'type':i.get('type'),'url':i.get('url'),'text':i.get('text')})
        MESHlist.append('MESH:'+i.get('MeSHUniqueID'))
        MESHurl.append(i.get('url'))
    else:
        notchemicallist.append(i)

In [722]:
len(notchemicallist)

0

In [181]:
chemical2classdict={}
count=0
for i in chemicallist:
    if not chemical2classdict.get(i.get('MESH')):
        chemical2classdict[i.get('MESH')]=[i.get('type')]
    else:
        if i.get('type') in chemical2classdict.get(i.get('MESH')):
            pass
        else:
            chemical2classdict[i.get('MESH')]=chemical2classdict.get(i.get('MESH'))+[i.get('type')]
            count+=1

In [182]:
chemical2subclassdict={}
for i in chemicallist:
    chemical2subclassdict[i.get('MESH')]=i.get('text')

In [183]:
chemical2urldict={}
for i in chemicallist:
    chemical2urldict[i.get('MESH')]=i.get('url')

In [184]:
for i in chemical_raw:
    i['PMID']=i.get('id')
    i['word']=i.get('text')
    del i['id']
    del i['text']

In [760]:
chemical_final=[]
chemical_mid=[]
chemical_dropout=[]

In [761]:
multi_chemical_type={}
for i in chemical_raw:
    if chemical2classdict.get(i.get('identifier')):
        chemical_final.append(dict(i,**{'class':chemical2classdict.get(i.get('identifier')),'subclass':chemical2subclassdict.get(i.get('identifier')),'url':chemical2urldict.get(i.get('identifier'))}))
        for j in chemical2classdict.get(i.get('identifier')):
            if len(chemical2classdict.get(i.get('identifier')))>1:
                if multi_chemical_type.get(chemical2subclassdict.get(i.get('identifier'))):
                    if j not in multi_chemical_type.get(chemical2subclassdict.get(i.get('identifier'))):
                        multi_chemical_type[chemical2subclassdict.get(i.get('identifier'))]=multi_chemical_type.get(chemical2subclassdict.get(i.get('identifier')))+[j]
                else:
                    multi_chemical_type[chemical2subclassdict.get(i.get('identifier'))]=[j]
    else:
        chemical_dropout.append(i)

In [762]:
# 1012
len(chemical_final)

1012

In [763]:
multi_chemical_type

{'Vancomycin': ['Peptide', 'Carbohydrate'],
 'Ceramides': ['Lipid', 'Carbohydrate'],
 'Angiotensins': ['Peptide', 'Protein'],
 'Lipopolysaccharides': ['Toxin', 'Lipid', 'Carbohydrate'],
 'Sphingomyelins': ['Lipid', 'Carbohydrate'],
 'Bleomycin': ['Peptide', 'Carbohydrate'],
 'O Antigens': ['Toxin', 'Lipid', 'Carbohydrate']}

for i in chemical_raw:
    if i.get('identifier') in chemical2classdict.keys():
        chemical_final.append(dict(i,**{'class':chemical2classdict.get(i.get('identifier')),'subclass':chemical2subclassdict.get(i.get('identifier')),'url':chemical2urldict.get(i.get('identifier'))}))
    else:
        chemical_dropout.append(i)

In [764]:
len(chemical_dropout)

6433

In [765]:
for j in pairs:
    names[j.replace(' ','_')+'list']=[]
    for i in chemical_final:
        if j in i.get('class'):
            names[j.replace(' ','_')+'list'].append(i)
    print(len(names[j.replace(' ','_')+'list']))

15
62
482
96
409
0


In [766]:
15+62+482+96+409

1064

In [767]:
chemical_deduplicated=set()
chemical_deduplicated_count=0
for j in pairs:
    names[j.replace(' ','_')+'list_raw']=[]
    for i in names[j.replace(' ','_')+'list']:
        k=i.get('PMID')
        i['DP']=rawpickwithAB.get(k).get('DP')
        i['AB']=rawpickwithAB.get(k).get('AB')
        i['URL']='https://pubmed.ncbi.nlm.nih.gov/'+k
        i['TI']=rawpickwithAB.get(k).get('TI')
        i['entity']=i.get('subclass')
        i['target']=i.get('word')
        i['type']=j
        i['JT']=rawpickwithAB.get(k).get('JT')
        if i.get('identifier') and re.match(r'MESH:',i.get('identifier')):
            i['MESH']=i.get('identifier')[5:]
        else:
            i['MESH']=''
        # tuples=(i.get("PMID"),i.get("entity"),i.get("word"),i.get("sentence"))
        tuples=(i.get("PMID"),i.get("entity"),i.get("word"),i.get("sentence"),i.get("type"))
        if tuples not in chemical_deduplicated:
            chemical_deduplicated.add(tuples)
            names[j.replace(' ','_')+'list_raw'].append(i)
        else:
            chemical_deduplicated_count+=1
            continue

In [768]:
chemical2ncbi={}
c=0
for j in pairs:
    for i in names[j.replace(' ','_')+'list_raw']:
        chemical2ncbi[i.get('url')]=''
        c+=1

In [769]:
# 124
len(chemical2ncbi)

124

In [193]:
chemical_converter=pickle.load(open('../ncbi/Chemical_converter.pkl','rb'))

In [194]:
len(chemical_converter)

475

In [195]:
import time
import datetime
from multiprocessing import Process, Manager, Pool
import os, time, random
from lxml import etree
import requests
def get_chemical_info(i,final_chemicals_inte):
    headers = {
        "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 \
        (KHTML, like Gecko) Chrome/83.0.4103.97 Safari/537.36"
    }
    url = 'https://www.ncbi.nlm.nih.gov' + i
    
    res=requests.get(url=url,headers=headers)
    html = etree.HTML(res.content)
    ncbisummary={
    'Summary' : ''.join(html.xpath('//*[@class="mesh_ds_scope_note"]/text()')),
    'YearIntroduced' : ''.join(html.xpath('//*[@class="mesh_year_introduced"]/text()'))[17:21],
    }
    final_chemicals_inte.update({i:ncbisummary})
    time.sleep(0.5)

In [770]:
judge=False
p = Pool(10)
manager=Manager()
final_chemicals_inte = manager.dict()
for i in chemical2ncbi.keys():
    if not chemical_converter.get(i):
        p.apply_async(get_chemical_info, args=(i,final_chemicals_inte))
        judge=True
p.close()
p.join()
if judge:
    chemical_converter.update(dict(final_chemicals_inte))
    pickle.dump(chemical_converter, open('../ncbi/Chemical_converter.pkl','wb'))

In [771]:
len(chemical_converter)

475

In [772]:
pairs

['Toxin',
 'Peptide',
 'Lipid',
 'Protein',
 'Carbohydrate',
 'Pharmaceutical Preparations']

In [773]:
for j in pairs:
    for i in range(len(names[j.replace(' ','_')+'list_raw'])):
        names[j.replace(' ','_')+'list_raw'][i]=dict(names[j.replace(' ','_')+'list_raw'][i],**chemical_converter.get(names[j.replace(' ','_')+'list_raw'][i].get('url')))

In [774]:
import time
import datetime
import re
dropout=[]
months=['Jan','Feb','Mar','Apr','May','Jun','Jul','Aug','Sep','Oct','Nov','Dec']
import re
for k in pairs:
    for i in names[k.replace(' ','_')+'list_raw']:
        j=i.get('DP')
        if re.search('[S,s]upplement',j,flags=re.M|re.I):
            j=re.sub('[S,s]upplement.*','',j,flags=re.M|re.I)
        j=re.sub('\s?-\s?','-',j)
        j=re.sub('\.','',j)
        j=re.sub('Aout','Aug',j)
        if len(j.split(' '))==3:
            try:
                date=int(datetime.datetime.strftime(datetime.datetime.strptime(j,'%Y %b %d'), '%Y%m%d'))
            except:
                if j.split()[1] in months:
                    if re.search(r'\d+',j.split()[2],flags=re.M|re.I):
                        j=j.split()[0]+' '+j.split()[1]+' '+ re.search(r'\d+',j.split()[2],flags=re.M|re.I).group()
                        try:
                            date=int(datetime.datetime.strftime(datetime.datetime.strptime(j,'%Y %b %d'), '%Y%m%d'))
                        except:
                            j=j[:4]
                            date=int(datetime.datetime.strftime(datetime.datetime.strptime(j,'%Y'), '%Y'+'0101'))
                            print('***************************************ERROR**********************************************')
                    else:
                        j=j.split()[0]+' '+j.split()[1]+' '+ '1'
                        try:
                            date=int(datetime.datetime.strftime(datetime.datetime.strptime(j,'%Y %b %d'), '%Y%m%d'))
                        except:
                            print(j)
                            j=j[:4]
                            date=int(datetime.datetime.strftime(datetime.datetime.strptime(j,'%Y'), '%Y'+'0101'))
                elif j.split()[1] not in months:
                    if re.search(r'([a-zA-Z]+)[-/]([a-zA-Z]+)',j.split()[1],flags=re.M|re.I) or re.search(r'([a-zA-Z]+)[-/](\d+)',j.split()[1],flags=re.M|re.I):
                        if re.search(r'([a-zA-Z]+)[-/]([a-zA-Z]+)',j.split()[1],flags=re.M|re.I):
                            middle=re.search(r'([a-zA-Z]+)[-/]([a-zA-Z]+)',j.split()[1],flags=re.M|re.I).group(1)
                        else:
                            middle=re.search(r'([a-zA-Z]+)[-/](\d+)',j.split()[1],flags=re.M|re.I).group(1)
                        if middle in months:
                            if re.search(r'\d+',j.split()[2],flags=re.M|re.I):
                                j=j.split()[0]+' '+middle+' '+ re.search(r'\d+',j.split()[2],flags=re.M|re.I).group()
                            else:
                                j=j.split()[0]+' '+middle+' '+ '1'
                        elif re.match(r'Sum(mer)?',middle):
                            if re.search(r'\d+',j.split()[2],flags=re.M|re.I):
                                j=j.split()[0]+' '+'Jun'+' '+ re.search(r'\d+',j.split()[2],flags=re.M|re.I).group()
                            else:
                                j=j.split()[0]+' '+'Jun'+' '+ '1'
                        elif re.match(r'Spr(ing)?',middle):
                            if re.search(r'\d+',j.split()[2],flags=re.M|re.I):
                                j=j.split()[0]+' '+'Mar'+' '+ re.search(r'\d+',j.split()[2],flags=re.M|re.I).group()
                            else:
                                j=j.split()[0]+' '+'Mar'+' '+ '1'
                        elif re.match(r'Win(ter)?',middle):
                            if re.search(r'\d+',j.split()[2],flags=re.M|re.I):
                                j=j.split()[0]+' '+'Dec'+' '+ re.search(r'\d+',j.split()[2],flags=re.M|re.I).group()
                            else:
                                j=j.split()[0]+' '+'Dec'+' '+ '1'    
                        elif re.match(r'Aut(umn)?',middle) or \
                        re.match(r'Fall',middle):
                            if re.search(r'\d+',j.split()[2],flags=re.M|re.I):
                                j=j.split()[0]+' '+'Sep'+' '+ re.search(r'\d+',j.split()[2],flags=re.M|re.I).group()
                            else:
                                j=j.split()[0]+' '+'Sep'+' '+ '1'      
                        try:
                            date=int(datetime.datetime.strftime(datetime.datetime.strptime(j,'%Y %b %d'), '%Y%m%d'))
                        except:
                            print(j)
                            j=j[:4]
                            date=int(datetime.datetime.strftime(datetime.datetime.strptime(j,'%Y'), '%Y'+'0101'))
                    else:
                        j=j[:4]
                        date=int(datetime.datetime.strftime(datetime.datetime.strptime(j,'%Y'), '%Y'+'0101'))
        elif len(j.split())==2:
            if re.match(r'Sum(mer)?',j.split()[1]):
                date=int(j[:4]+'0601')
            elif re.match(r'Spr(ing)?',j.split()[1]):
                date=int(j[:4]+'0301')
            elif re.match(r'Win(ter)?',j.split()[1]):
                date=int(j[:4]+'1201')
            elif re.match(r'Aut(umn)?',j.split()[1]) or re.match(r'Fall',j.split()[1]):
                date=int(j[:4]+'0901')
            elif re.match(r'Sum(mer)?',j.split()[0]):
                date=int(j[-4:]+'0601')
            elif re.match(r'Spr(ing)?',j.split()[0]):
                date=int(j[-4:]+'0301')
            elif re.match(r'Win(ter)?',j.split()[0]):
                date=int(j[-4:]+'1201')
            elif re.match(r'Aut(umn)?',j.split()[0]) or re.match(r'Fall',j.split()[0]):
                date=int(j[-4:]+'0901')
            else:
                try:
                    date=int(datetime.datetime.strftime(datetime.datetime.strptime(j,'%Y %b'), '%Y%m'+'01'))
                except:
                    if re.search(r'([a-zA-Z]+)[-/]([a-zA-Z]+)',j.split()[1],flags=re.M|re.I) or re.search(r'([a-zA-Z]+)[-/](\d+)',j.split()[1],flags=re.M|re.I):
                        if re.search(r'([a-zA-Z]+)[-/]([a-zA-Z]+)',j.split()[1],flags=re.M|re.I):
                            middle=re.search(r'([a-zA-Z]+)[-/]([a-zA-Z]+)',j.split()[1],flags=re.M|re.I).group(1)
                        else:
                            middle=re.search(r'([a-zA-Z]+)[-/](\d+)',j.split()[1],flags=re.M|re.I).group(1)
                        if middle in months:
                            j=j.split()[0]+' '+middle+' '+ '1'
                        elif re.match(r'Sum(mer)?',middle):
                            j=j.split()[0]+' '+'Jun'+' '+ '1'
                        elif re.match(r'Spr(ing)?',middle):
                            j=j.split()[0]+' '+'Mar'+' '+ '1'
                        elif re.match(r'Win(ter)?',middle):
                            j=j.split()[0]+' '+'Dec'+' '+ '1'    
                        elif re.match(r'Aut(umn)?',middle) or \
                        re.match(r'Fall',middle):
                            j=j.split()[0]+' '+'Sep'+' '+ '1'      
                        try:
                            date=int(datetime.datetime.strftime(datetime.datetime.strptime(j,'%Y %b %d'), '%Y%m%d'))
                        except:
                            j=j[:4]
                            date=int(datetime.datetime.strftime(datetime.datetime.strptime(j,'%Y'), '%Y'+'0101'))
                    elif re.search(r'(\d+)[-/](\d+)',j.split()[1],flags=re.M|re.I):
                        midd=re.search(r'(\d+)[-/](\d+)',j.split()[1],flags=re.M|re.I).group(1)
                        if len(midd)!=2:
                            midd='0'+midd
                        j=j[:4]
                        date=int(datetime.datetime.strftime(datetime.datetime.strptime(j,'%Y'), '%Y'+midd+'01'))
                    else:
                        print(j)
                        j=j[:4]
                        date=int(datetime.datetime.strftime(datetime.datetime.strptime(j,'%Y'), '%Y'+'0101'))
        elif len(j.split())==1:
            j=j[:4]
            date=int(datetime.datetime.strftime(datetime.datetime.strptime(j,'%Y'), '%Y'+'0101'))
        else:
            j=j.split()[0]+' '+j.split()[1]+' '+re.search(r'\d+',j.split()[2],flags=re.M|re.I).group()
            date=int(datetime.datetime.strftime(datetime.datetime.strptime(j,'%Y %b %d'), '%Y%m%d'))
        i['date']=date
        i['year']=int(str(i.get('date'))[:4])

In [775]:
for k in pairs:
    names[k.replace(' ','_')+'_raw']=copy.deepcopy(names[k.replace(' ','_')+'list_raw'])
    for i in names[k.replace(' ','_')+'_raw']:
        i['addtime']=now_time

# Carbohydrate_raw

In [105]:
collections_Carbohydrate_raw=db.Carbohydrate_raw

In [777]:
len(Carbohydrate_raw)

381

In [778]:
Carbohydrate_raw[0]

{'AB': 'The world is moving towards a healthier lifestyle where people are changing their eating habits, which influenced edible rose flowers to emerge as a pioneer in the field of nutraceutical and food industries. Roses are a good source of dietary phytochemicals viz., flavonoids (anthocyanins, flavonols, and flavonols), carotenoids, and phenolic acids. The presence of such phytochemicals makes rose as an anti-oxidant, anti-inflammatory, anti-cancerous, anti-aging, anti-microbial, hepatoprotective, and neurogenic agent. Historically edible rose flowers have been used in the preparation of traditional food products and delicacies such as gulkand, punkhuri, and rose petal tea and have found application in traditional medicine such as Ayurveda to treat hyperacidity, vata, pitta, constipation, abdominal pains, and various other illnesses. Over a period of time, concept of edible flowers has gotten more recognition and now roses are used in the preparation of many food products such as ja

In [779]:
if len(Carbohydrate_raw)!=0:
    collections_Carbohydrate_raw.insert_many(Carbohydrate_raw)

In [709]:
shenme=list(collections_Carbohydrate_raw.find())

In [715]:
for i in shenme:
    if i.get('PMID')=='35302400':
        print('i')
        break

In [714]:
shenme[-1]

{'_id': ObjectId('639369f87aeeb1f33fdc493d'),
 'AB': 'OBJECTIVE: To investigate non-urological patients with multiple comorbidities for factors contributing towards differences in testosterone concentration in multi-ethnic Malaysian men. METHODS: Sexually active men >=40 years with no known urological problems were recruited at the phlebotomy clinic at our center. A brief history along with latest fasting lipid profile and plasma glucose levels were obtained. An Aging Male Symptoms (AMS) questionnaire was administered; waist circumference (WC) and serum testosterone concentration were measured. Analysis of testosterone concentration between Malay, Indian and Chinese men was performed. Statistical tests such as ANOVA, Chi square test, univariate and multivariable regression were performed. Any P<0.05 denoted as statistically significant. RESULTS: Among the 604 participants analyzed, mean testosterone concentration was significantly lower in Malays (15.1+-5.9 nmol/L) compared to the Chin

# Carbohydrate_all

In [429]:
collections_Carbohydrate_all=db.Carbohydrate_all

collections_Carbohydrate_all.remove({'addtime':now_time})

In [579]:
Carbohydrate_raw_all=list(collections_Carbohydrate_raw.find())

In [431]:
len(Carbohydrate_raw_all)

17390

In [432]:
for i in Carbohydrate_raw_all:
    i['type']='Carbohydrate'
    i["PMID"]=[i.get("PMID")]
    i["sentence"]=[[i.get("sentence")]]
    pmid=i.get('PMID')[0]
    i['DP']=[i.get("DP")]
    i['AB']=[i.get("AB")]
    i['URL']='https://pubmed.ncbi.nlm.nih.gov/'+pmid
    i['TI']=[i.get("TI")]
    i['target']=[[i.get('word')]]
    i["date"]=[i.get("date")]
    i["year"]=[i.get("year")]
    i['JT']=[i.get('JT')]

In [433]:
Carbohydrate_mid={}
for i in Carbohydrate_raw_all:
    if not Carbohydrate_mid.get(i.get("entity")):
        Carbohydrate_mid[i.get("entity")]=i
    else:
        if i.get('PMID')[0] in Carbohydrate_mid.get(i.get('entity')).get('PMID'):

            ind=Carbohydrate_mid.get(i.get('entity')).get('PMID').index(i.get('PMID')[0])
            if i.get("sentence")[0][0] in Carbohydrate_mid.get(i.get('entity')).get('sentence')[ind]:
                continue
            else:
                Carbohydrate_mid[i.get('entity')]['sentence'][ind] = Carbohydrate_mid.get(i.get('entity')).get('sentence')[ind] + i.get("sentence")[0]
                Carbohydrate_mid[i.get('entity')]['target'][ind] = Carbohydrate_mid.get(i.get('entity')).get('target')[ind] + i.get("target")[0]

        else:
            Carbohydrate_mid[i.get("entity")]["PMID"]=Carbohydrate_mid.get(i.get("entity")).get("PMID")+i.get("PMID")
            Carbohydrate_mid[i.get("entity")]["sentence"]=Carbohydrate_mid.get(i.get("entity")).get("sentence")+i.get("sentence")
            Carbohydrate_mid[i.get("entity")]["AB"]=Carbohydrate_mid.get(i.get("entity")).get("AB")+i.get("AB")
            Carbohydrate_mid[i.get("entity")]["DP"]=Carbohydrate_mid.get(i.get("entity")).get("DP")+i.get("DP")
            Carbohydrate_mid[i.get("entity")]["TI"]=Carbohydrate_mid.get(i.get("entity")).get("TI")+i.get("TI")
            Carbohydrate_mid[i.get("entity")]["target"]=Carbohydrate_mid.get(i.get("entity")).get("target")+i.get("target")
            Carbohydrate_mid[i.get("entity")]["date"]=Carbohydrate_mid.get(i.get("entity")).get("date")+i.get("date")
            Carbohydrate_mid[i.get("entity")]["year"]=Carbohydrate_mid.get(i.get("entity")).get("year")+i.get("year")
            Carbohydrate_mid[i.get("entity")]["JT"]=Carbohydrate_mid.get(i.get("entity")).get("JT")+i.get("JT")

In [434]:
Carbohydrate_all=list(Carbohydrate_mid.values())

In [435]:
len(Carbohydrate_all)

202

In [436]:
for i in Carbohydrate_all:
    i['allyears']=[]
    i['datestring']=[]
    for j in i.get('date'):
        if int(str(j)[:4]) not in i.get('allyears'):
            i['allyears']= i.get('allyears')+[int(str(j)[:4])]
    for j in i.get('date'):
        if str(j)[:4]+"-"+str(j)[4:6]+"-"+str(j)[6:] not in i.get('datestring'):
            i['datestring']=i.get('datestring')+[str(j)[:4]+"-"+str(j)[4:6]+"-"+str(j)[6:]]
    i['allyears']=sorted(i.get('allyears'), reverse=True)
    i['datestring']=sorted(i.get('datestring'), reverse=True)
    upperlist=set()
    for j in sum(i.get('target'),[]):
        upperlist.add(j.upper())
    i['targetshapes']=list(upperlist)
    i['addtime']=now_time

In [437]:
Carbohydrate_all[10].get('Summary')

'A class of sphingolipids found largely in the brain and other nervous tissue. They contain phosphocholine or phosphoethanolamine as their polar head group so therefore are the only sphingolipids classified as PHOSPHOLIPIDS.'

In [789]:
# collections_Carbohydrate_all.insert_many(Carbohydrate_all)

# Lipid_raw

In [114]:
collections_Lipid_raw=db.Lipid_raw

In [793]:
len(Lipid_raw)

455

In [794]:
if len(Lipid_raw)!=0:
    collections_Lipid_raw.insert_many(Lipid_raw)

# Lipid_all

In [795]:
collections_Lipid_all=db.Lipid_all

collections_Lipid_all.remove({'addtime':now_time})

In [584]:
Lipid_raw_all=list(collections_Lipid_raw.find())

In [589]:
len(Lipid_raw_all)

18859

In [500]:
for i in Lipid_raw_all:
    if i.get('entity')=='Angiotensins':
        print(i)

In [447]:
for i in Lipid_raw_all:
    i['type']='Lipid'
    i["PMID"]=[i.get("PMID")]
    i["sentence"]=[[i.get("sentence")]]
    pmid=i.get('PMID')[0]
    i['DP']=[i.get("DP")]
    i['AB']=[i.get("AB")]
    i['URL']='https://pubmed.ncbi.nlm.nih.gov/'+pmid
    i['TI']=[i.get("TI")]
    i['target']=[[i.get('word')]]
    i["date"]=[i.get("date")]
    i["year"]=[i.get("year")]
    i['JT']=[i.get('JT')]

In [448]:
Lipid_mid={}
for i in Lipid_raw_all:
    if not Lipid_mid.get(i.get("entity")):
        Lipid_mid[i.get("entity")]=i
    else:
        if i.get('PMID')[0] in Lipid_mid.get(i.get('entity')).get('PMID'):

            ind=Lipid_mid.get(i.get('entity')).get('PMID').index(i.get('PMID')[0])
            if i.get("sentence")[0][0] in Lipid_mid.get(i.get('entity')).get('sentence')[ind]:
                continue
            else:
                Lipid_mid[i.get('entity')]['sentence'][ind] = Lipid_mid.get(i.get('entity')).get('sentence')[ind] + i.get("sentence")[0]
                Lipid_mid[i.get('entity')]['target'][ind] = Lipid_mid.get(i.get('entity')).get('target')[ind] + i.get("target")[0]

        else:
            Lipid_mid[i.get("entity")]["PMID"]=Lipid_mid.get(i.get("entity")).get("PMID")+i.get("PMID")
            Lipid_mid[i.get("entity")]["sentence"]=Lipid_mid.get(i.get("entity")).get("sentence")+i.get("sentence")
            Lipid_mid[i.get("entity")]["AB"]=Lipid_mid.get(i.get("entity")).get("AB")+i.get("AB")
            Lipid_mid[i.get("entity")]["DP"]=Lipid_mid.get(i.get("entity")).get("DP")+i.get("DP")
            Lipid_mid[i.get("entity")]["TI"]=Lipid_mid.get(i.get("entity")).get("TI")+i.get("TI")
            Lipid_mid[i.get("entity")]["target"]=Lipid_mid.get(i.get("entity")).get("target")+i.get("target")
            Lipid_mid[i.get("entity")]["date"]=Lipid_mid.get(i.get("entity")).get("date")+i.get("date")
            Lipid_mid[i.get("entity")]["year"]=Lipid_mid.get(i.get("entity")).get("year")+i.get("year")
            Lipid_mid[i.get("entity")]["JT"]=Lipid_mid.get(i.get("entity")).get("JT")+i.get("JT")

In [449]:
Lipid_all=list(Lipid_mid.values())

In [450]:
len(Lipid_all)

190

In [451]:
for i in Lipid_all:
    i['allyears']=[]
    i['datestring']=[]
    for j in i.get('date'):
        if int(str(j)[:4]) not in i.get('allyears'):
            i['allyears']= i.get('allyears')+[int(str(j)[:4])]
    for j in i.get('date'):
        if str(j)[:4]+"-"+str(j)[4:6]+"-"+str(j)[6:] not in i.get('datestring'):
            i['datestring']=i.get('datestring')+[str(j)[:4]+"-"+str(j)[4:6]+"-"+str(j)[6:]]
    i['allyears']=sorted(i.get('allyears'), reverse=True)
    i['datestring']=sorted(i.get('datestring'), reverse=True)
    upperlist=set()
    for j in sum(i.get('target'),[]):
        upperlist.add(j.upper())
    i['targetshapes']=list(upperlist)
    i['addtime']=now_time

In [803]:
# collections_Lipid_all.insert_many(Lipid_all)

# Peptide_raw

In [122]:
collections_Peptide_raw=db.Peptide_raw

In [805]:
len(Peptide_raw)

55

In [806]:
if len(Peptide_raw)!=0:
    collections_Peptide_raw.insert_many(Peptide_raw)

# Peptide_all

In [405]:
collections_Peptide_all=db.Peptide_all

collections_Peptide_all.remove({'addtime':now_time})

In [591]:
Peptide_raw_all=list(collections_Peptide_raw.find())

In [595]:
len(Peptide_raw_all)

3345

In [504]:
for i in Peptide_raw_all:
    if i.get('entity')=='Angiotensins':
        print(i)
        break

{'_id': ObjectId('63fa3ff3648b16758fbbf424'), 'AB': 'AIMS: Real-world evidence about heart failure (HF) medical care for elderly patients is limited. The aim of this study was to clarify the age-dependent associations of HF medications at discharge with clinical outcomes in a super-aged society using a real-world big database. METHODS AND RESULTS: Patients hospitalized with a first episode of acute HF in 2013-2019 were identified in a nationwide claims database in Japan and were analyzed based on the HF medications at discharge. Hazard ratios (HRs) and 95 % confidence intervals (95 % CIs) for clinical outcomes were calculated after adjustment. Among 325,468 patients from 4351 hospitals, 130,230 (40.0 %) were aged >= 85 years. All-cause mortality and HF readmission rate within 1 year were 14 % and 23 %, respectively. beta-Blockers and angiotensin-converting-enzyme inhibitors or angiotensin II receptor blockers (ACEI/ARB) uses were associated with lower mortality (HR [95 %CI]: 0.84 [0.83

In [505]:
for i in Peptide_raw_all:
    i['type']='Peptide'
    i["PMID"]=[i.get("PMID")]
    i["sentence"]=[[i.get("sentence")]]
    pmid=i.get('PMID')[0]
    i['DP']=[i.get("DP")]
    i['AB']=[i.get("AB")]
    i['URL']='https://pubmed.ncbi.nlm.nih.gov/'+pmid
    i['TI']=[i.get("TI")]
    i['target']=[[i.get('word')]]
    i["date"]=[i.get("date")]
    i["year"]=[i.get("year")]
    i['JT']=[i.get('JT')]

In [506]:
Peptide_mid={}
for i in Peptide_raw_all:
    if not Peptide_mid.get(i.get("entity")):
        Peptide_mid[i.get("entity")]=i
    else:
        if i.get('PMID')[0] in Peptide_mid.get(i.get('entity')).get('PMID'):

            ind=Peptide_mid.get(i.get('entity')).get('PMID').index(i.get('PMID')[0])
            if i.get("sentence")[0][0] in Peptide_mid.get(i.get('entity')).get('sentence')[ind]:
                continue
            else:
                Peptide_mid[i.get('entity')]['sentence'][ind] = Peptide_mid.get(i.get('entity')).get('sentence')[ind] + i.get("sentence")[0]
                Peptide_mid[i.get('entity')]['target'][ind] = Peptide_mid.get(i.get('entity')).get('target')[ind] + i.get("target")[0]

        else:
            Peptide_mid[i.get("entity")]["PMID"]=Peptide_mid.get(i.get("entity")).get("PMID")+i.get("PMID")
            Peptide_mid[i.get("entity")]["sentence"]=Peptide_mid.get(i.get("entity")).get("sentence")+i.get("sentence")
            Peptide_mid[i.get("entity")]["AB"]=Peptide_mid.get(i.get("entity")).get("AB")+i.get("AB")
            Peptide_mid[i.get("entity")]["DP"]=Peptide_mid.get(i.get("entity")).get("DP")+i.get("DP")
            Peptide_mid[i.get("entity")]["TI"]=Peptide_mid.get(i.get("entity")).get("TI")+i.get("TI")
            Peptide_mid[i.get("entity")]["target"]=Peptide_mid.get(i.get("entity")).get("target")+i.get("target")
            Peptide_mid[i.get("entity")]["date"]=Peptide_mid.get(i.get("entity")).get("date")+i.get("date")
            Peptide_mid[i.get("entity")]["year"]=Peptide_mid.get(i.get("entity")).get("year")+i.get("year")
            Peptide_mid[i.get("entity")]["JT"]=Peptide_mid.get(i.get("entity")).get("JT")+i.get("JT")

In [507]:
Peptide_all=list(Peptide_mid.values())

In [508]:
len(Peptide_all)

74

In [509]:
for i in Peptide_all:
    i['allyears']=[]
    i['datestring']=[]
    for j in i.get('date'):
        if int(str(j)[:4]) not in i.get('allyears'):
            i['allyears']= i.get('allyears')+[int(str(j)[:4])]
    for j in i.get('date'):
        if str(j)[:4]+"-"+str(j)[4:6]+"-"+str(j)[6:] not in i.get('datestring'):
            i['datestring']=i.get('datestring')+[str(j)[:4]+"-"+str(j)[4:6]+"-"+str(j)[6:]]
    i['allyears']=sorted(i.get('allyears'), reverse=True)
    i['datestring']=sorted(i.get('datestring'), reverse=True)
    upperlist=set()
    for j in sum(i.get('target'),[]):
        upperlist.add(j.upper())
    i['targetshapes']=list(upperlist)
    i['addtime']=now_time

In [815]:
# collections_Peptide_all.insert_many(Peptide_all)

# Pharmaceutical_Preparations_raw

In [130]:
collections_Pharmaceutical_Preparations_raw=db.Pharmaceutical_Preparations_raw

In [817]:
len(Pharmaceutical_Preparations_raw)

0

In [818]:
if len(Pharmaceutical_Preparations_raw)!=0:
    collections_Pharmaceutical_Preparations_raw.insert_many(Pharmaceutical_Preparations_raw)

# Pharmaceutical_Preparations_all

In [819]:
collections_Pharmaceutical_Preparations_all=db.Pharmaceutical_Preparations_all

collections_Pharmaceutical_Preparations_all.remove({'addtime':now_time})

In [596]:
Pharmaceutical_Preparations_raw_all=list(collections_Pharmaceutical_Preparations_raw.find())

In [600]:
len(Pharmaceutical_Preparations_raw_all)

267

In [463]:
for i in Pharmaceutical_Preparations_raw_all:
    i['type']='Pharmaceutical Preparations'
    i["PMID"]=[i.get("PMID")]
    i["sentence"]=[[i.get("sentence")]]
    pmid=i.get('PMID')[0]
    i['DP']=[i.get("DP")]
    i['AB']=[i.get("AB")]
    i['URL']='https://pubmed.ncbi.nlm.nih.gov/'+pmid
    i['TI']=[i.get("TI")]
    i['target']=[[i.get('word')]]
    i["date"]=[i.get("date")]
    i["year"]=[i.get("year")]
    i['JT']=[i.get('JT')]

In [464]:
Pharmaceutical_Preparations_mid={}
for i in Pharmaceutical_Preparations_raw_all:
    if not Pharmaceutical_Preparations_mid.get(i.get("entity")):
        Pharmaceutical_Preparations_mid[i.get("entity")]=i
    else:
        if i.get('PMID')[0] in Pharmaceutical_Preparations_mid.get(i.get('entity')).get('PMID'):

            ind=Pharmaceutical_Preparations_mid.get(i.get('entity')).get('PMID').index(i.get('PMID')[0])
            if i.get("sentence")[0][0] in Pharmaceutical_Preparations_mid.get(i.get('entity')).get('sentence')[ind]:
                continue
            else:
                Pharmaceutical_Preparations_mid[i.get('entity')]['sentence'][ind] = Pharmaceutical_Preparations_mid.get(i.get('entity')).get('sentence')[ind] + i.get("sentence")[0]
                Pharmaceutical_Preparations_mid[i.get('entity')]['target'][ind] = Pharmaceutical_Preparations_mid.get(i.get('entity')).get('target')[ind] + i.get("target")[0]

        else:
            Pharmaceutical_Preparations_mid[i.get("entity")]["PMID"]=Pharmaceutical_Preparations_mid.get(i.get("entity")).get("PMID")+i.get("PMID")
            Pharmaceutical_Preparations_mid[i.get("entity")]["sentence"]=Pharmaceutical_Preparations_mid.get(i.get("entity")).get("sentence")+i.get("sentence")
            Pharmaceutical_Preparations_mid[i.get("entity")]["AB"]=Pharmaceutical_Preparations_mid.get(i.get("entity")).get("AB")+i.get("AB")
            Pharmaceutical_Preparations_mid[i.get("entity")]["DP"]=Pharmaceutical_Preparations_mid.get(i.get("entity")).get("DP")+i.get("DP")
            Pharmaceutical_Preparations_mid[i.get("entity")]["TI"]=Pharmaceutical_Preparations_mid.get(i.get("entity")).get("TI")+i.get("TI")
            Pharmaceutical_Preparations_mid[i.get("entity")]["target"]=Pharmaceutical_Preparations_mid.get(i.get("entity")).get("target")+i.get("target")
            Pharmaceutical_Preparations_mid[i.get("entity")]["date"]=Pharmaceutical_Preparations_mid.get(i.get("entity")).get("date")+i.get("date")
            Pharmaceutical_Preparations_mid[i.get("entity")]["year"]=Pharmaceutical_Preparations_mid.get(i.get("entity")).get("year")+i.get("year")
            Pharmaceutical_Preparations_mid[i.get("entity")]["JT"]=Pharmaceutical_Preparations_mid.get(i.get("entity")).get("JT")+i.get("JT")

In [465]:
Pharmaceutical_Preparations_all=list(Pharmaceutical_Preparations_mid.values())

In [466]:
len(Pharmaceutical_Preparations_all)

10

In [467]:
for i in Pharmaceutical_Preparations_all:
    i['allyears']=[]
    i['datestring']=[]
    for j in i.get('date'):
        if int(str(j)[:4]) not in i.get('allyears'):
            i['allyears']= i.get('allyears')+[int(str(j)[:4])]
    for j in i.get('date'):
        if str(j)[:4]+"-"+str(j)[4:6]+"-"+str(j)[6:] not in i.get('datestring'):
            i['datestring']=i.get('datestring')+[str(j)[:4]+"-"+str(j)[4:6]+"-"+str(j)[6:]]
    i['allyears']=sorted(i.get('allyears'), reverse=True)
    i['datestring']=sorted(i.get('datestring'), reverse=True)
    upperlist=set()
    for j in sum(i.get('target'),[]):
        upperlist.add(j.upper())
    i['targetshapes']=list(upperlist)
    i['addtime']=now_time

In [827]:
# collections_Pharmaceutical_Preparations_all.insert_many(Pharmaceutical_Preparations_all)

# Protein_raw

In [138]:
collections_Protein_raw=db.Protein_raw

In [829]:
len(Protein_raw)

77

In [830]:
if len(Protein_raw)!=0:
    collections_Protein_raw.insert_many(Protein_raw)

# Protein_all

In [409]:
collections_Protein_all=db.Protein_all

collections_Protein_all.remove({'addtime':now_time})

In [601]:
Protein_raw_all=list(collections_Protein_raw.find())

In [605]:
len(Protein_raw_all)

1828

In [470]:
for i in Protein_raw_all:
    if i.get('entity')=='Angiotensins':
        print(i)


{'_id': ObjectId('63fa4011648b16758fbbfc27'), 'AB': 'AIMS: Real-world evidence about heart failure (HF) medical care for elderly patients is limited. The aim of this study was to clarify the age-dependent associations of HF medications at discharge with clinical outcomes in a super-aged society using a real-world big database. METHODS AND RESULTS: Patients hospitalized with a first episode of acute HF in 2013-2019 were identified in a nationwide claims database in Japan and were analyzed based on the HF medications at discharge. Hazard ratios (HRs) and 95 % confidence intervals (95 % CIs) for clinical outcomes were calculated after adjustment. Among 325,468 patients from 4351 hospitals, 130,230 (40.0 %) were aged >= 85 years. All-cause mortality and HF readmission rate within 1 year were 14 % and 23 %, respectively. beta-Blockers and angiotensin-converting-enzyme inhibitors or angiotensin II receptor blockers (ACEI/ARB) uses were associated with lower mortality (HR [95 %CI]: 0.84 [0.83

In [471]:
for i in Protein_raw_all:
    i['type']='Protein'
    i["PMID"]=[i.get("PMID")]
    i["sentence"]=[[i.get("sentence")]]
    pmid=i.get('PMID')[0]
    i['DP']=[i.get("DP")]
    i['AB']=[i.get("AB")]
    i['URL']='https://pubmed.ncbi.nlm.nih.gov/'+pmid
    i['TI']=[i.get("TI")]
    i['target']=[[i.get('word')]]
    i["date"]=[i.get("date")]
    i["year"]=[i.get("year")]
    i['JT']=[i.get('JT')]

In [472]:
Protein_mid={}
for i in Protein_raw_all:
    if not Protein_mid.get(i.get("entity")):
        Protein_mid[i.get("entity")]=i
    else:
        if i.get('PMID')[0] in Protein_mid.get(i.get('entity')).get('PMID'):

            ind=Protein_mid.get(i.get('entity')).get('PMID').index(i.get('PMID')[0])
            if i.get("sentence")[0][0] in Protein_mid.get(i.get('entity')).get('sentence')[ind]:
                continue
            else:
                Protein_mid[i.get('entity')]['sentence'][ind] = Protein_mid.get(i.get('entity')).get('sentence')[ind] + i.get("sentence")[0]
                Protein_mid[i.get('entity')]['target'][ind] = Protein_mid.get(i.get('entity')).get('target')[ind] + i.get("target")[0]

        else:
            Protein_mid[i.get("entity")]["PMID"]=Protein_mid.get(i.get("entity")).get("PMID")+i.get("PMID")
            Protein_mid[i.get("entity")]["sentence"]=Protein_mid.get(i.get("entity")).get("sentence")+i.get("sentence")
            Protein_mid[i.get("entity")]["AB"]=Protein_mid.get(i.get("entity")).get("AB")+i.get("AB")
            Protein_mid[i.get("entity")]["DP"]=Protein_mid.get(i.get("entity")).get("DP")+i.get("DP")
            Protein_mid[i.get("entity")]["TI"]=Protein_mid.get(i.get("entity")).get("TI")+i.get("TI")
            Protein_mid[i.get("entity")]["target"]=Protein_mid.get(i.get("entity")).get("target")+i.get("target")
            Protein_mid[i.get("entity")]["date"]=Protein_mid.get(i.get("entity")).get("date")+i.get("date")
            Protein_mid[i.get("entity")]["year"]=Protein_mid.get(i.get("entity")).get("year")+i.get("year")
            Protein_mid[i.get("entity")]["JT"]=Protein_mid.get(i.get("entity")).get("JT")+i.get("JT")

In [473]:
Protein_all=list(Protein_mid.values())

In [474]:
len(Protein_all)

23

In [475]:
for i in Protein_all:
    i['allyears']=[]
    i['datestring']=[]
    for j in i.get('date'):
        if int(str(j)[:4]) not in i.get('allyears'):
            i['allyears']= i.get('allyears')+[int(str(j)[:4])]
    for j in i.get('date'):
        if str(j)[:4]+"-"+str(j)[4:6]+"-"+str(j)[6:] not in i.get('datestring'):
            i['datestring']=i.get('datestring')+[str(j)[:4]+"-"+str(j)[4:6]+"-"+str(j)[6:]]
    i['allyears']=sorted(i.get('allyears'), reverse=True)
    i['datestring']=sorted(i.get('datestring'), reverse=True)
    upperlist=set()
    for j in sum(i.get('target'),[]):
        upperlist.add(j.upper())
    i['targetshapes']=list(upperlist)
    i['addtime']=now_time

In [839]:
# collections_Protein_all.insert_many(Protein_all)

# Toxin_raw

In [146]:
collections_Toxin_raw=db.Toxin_raw

In [841]:
len(Toxin_raw)

14

In [842]:
if len(Toxin_raw)!=0:
    collections_Toxin_raw.insert_many(Toxin_raw)

# Toxin_all

In [843]:
collections_Toxin_all=db.Toxin_all

collections_Toxin_all.remove({'addtime':now_time})

In [476]:
Toxin_raw_all=list(collections_Toxin_raw.find())

In [609]:
len(Toxin_raw_all)

539

In [478]:
for i in Toxin_raw_all:
    i['type']='Toxin'
    i["PMID"]=[i.get("PMID")]
    i["sentence"]=[[i.get("sentence")]]
    pmid=i.get('PMID')[0]
    i['DP']=[i.get("DP")]
    i['AB']=[i.get("AB")]
    i['URL']='https://pubmed.ncbi.nlm.nih.gov/'+pmid
    i['TI']=[i.get("TI")]
    i['target']=[[i.get('word')]]
    i["date"]=[i.get("date")]
    i["year"]=[i.get("year")]
    i['JT']=[i.get('JT')]

In [479]:
Toxin_mid={}
for i in Toxin_raw_all:
    if not Toxin_mid.get(i.get("entity")):
        Toxin_mid[i.get("entity")]=i
    else:
        if i.get('PMID')[0] in Toxin_mid.get(i.get('entity')).get('PMID'):

            ind=Toxin_mid.get(i.get('entity')).get('PMID').index(i.get('PMID')[0])
            if i.get("sentence")[0][0] in Toxin_mid.get(i.get('entity')).get('sentence')[ind]:
                continue
            else:
                Toxin_mid[i.get('entity')]['sentence'][ind] = Toxin_mid.get(i.get('entity')).get('sentence')[ind] + i.get("sentence")[0]
                Toxin_mid[i.get('entity')]['target'][ind] = Toxin_mid.get(i.get('entity')).get('target')[ind] + i.get("target")[0]

        else:
            Toxin_mid[i.get("entity")]["PMID"]=Toxin_mid.get(i.get("entity")).get("PMID")+i.get("PMID")
            Toxin_mid[i.get("entity")]["sentence"]=Toxin_mid.get(i.get("entity")).get("sentence")+i.get("sentence")
            Toxin_mid[i.get("entity")]["AB"]=Toxin_mid.get(i.get("entity")).get("AB")+i.get("AB")
            Toxin_mid[i.get("entity")]["DP"]=Toxin_mid.get(i.get("entity")).get("DP")+i.get("DP")
            Toxin_mid[i.get("entity")]["TI"]=Toxin_mid.get(i.get("entity")).get("TI")+i.get("TI")
            Toxin_mid[i.get("entity")]["target"]=Toxin_mid.get(i.get("entity")).get("target")+i.get("target")
            Toxin_mid[i.get("entity")]["date"]=Toxin_mid.get(i.get("entity")).get("date")+i.get("date")
            Toxin_mid[i.get("entity")]["year"]=Toxin_mid.get(i.get("entity")).get("year")+i.get("year")
            Toxin_mid[i.get("entity")]["JT"]=Toxin_mid.get(i.get("entity")).get("JT")+i.get("JT")

In [480]:
Toxin_all=list(Toxin_mid.values())

In [481]:
len(Toxin_all)

18

In [482]:
for i in Toxin_all:
    i['allyears']=[]
    i['datestring']=[]
    for j in i.get('date'):
        if int(str(j)[:4]) not in i.get('allyears'):
            i['allyears']= i.get('allyears')+[int(str(j)[:4])]
    for j in i.get('date'):
        if str(j)[:4]+"-"+str(j)[4:6]+"-"+str(j)[6:] not in i.get('datestring'):
            i['datestring']=i.get('datestring')+[str(j)[:4]+"-"+str(j)[4:6]+"-"+str(j)[6:]]
    i['allyears']=sorted(i.get('allyears'), reverse=True)
    i['datestring']=sorted(i.get('datestring'), reverse=True)
    upperlist=set()
    for j in sum(i.get('target'),[]):
        upperlist.add(j.upper())
    i['targetshapes']=list(upperlist)
    i['addtime']=now_time

In [851]:
# collections_Toxin_all.insert_many(Toxin_all)

# Mutation_raw

In [154]:
collections_Mutation_raw=db.Mutation_raw

collections_Mutation_raw.remove({'addtime':now_time})

In [275]:
Mutation_pubtator_addpmid={}
for i in pubtator_raw:
    if i.get('type')=='Mutation':
        if Mutation_pubtator_addpmid.get(i.get('id')):
            Mutation_pubtator_addpmid[i.get('id')]=Mutation_pubtator_addpmid.get(i.get('id'))+[i]
        else:
            Mutation_pubtator_addpmid[i.get('id')]=[i]

In [276]:
Mutation_output_json=list(Mutation_pubtator_addpmid.values())

In [277]:
Mutation_raw=sum(Mutation_output_json,[])

In [278]:
len(Mutation_raw)

169

In [279]:
for i in Mutation_raw:
    i['PMID']=i.get('id')
    i['word']=i.get('text')
    del i['id']
    del i['text']

In [280]:
Mutation_deduplicated=set()
Mutation_deduplicated_count=0
mutation_list_raw=[]
for i in Mutation_raw:
    if i.get('identifier'):
        k=i.get('PMID')
        i['DP']=rawpickwithAB.get(k).get('DP')
        i['AB']=rawpickwithAB.get(k).get('AB')
        i['URL']='https://pubmed.ncbi.nlm.nih.gov/'+k
        i['TI']=rawpickwithAB.get(k).get('TI')
        i['entity']=i.get('identifier')
        i['target']=i.get('word')
        i['type']='Mutation'
        i['JT']=rawpickwithAB.get(k).get('JT')
        # tuples=(i.get("PMID"),i.get("entity"),i.get("word"),i.get("sentence"))
        tuples=(i.get("PMID"),i.get("entity"),i.get("word"),i.get("sentence"))
        if tuples not in Mutation_deduplicated:
            Mutation_deduplicated.add(tuples)
            mutation_list_raw.append(i)
        else:
            Mutation_deduplicated_count+=1
            continue

In [281]:
len(mutation_list_raw)

149

In [282]:
import re
dropout1=[]
dropout2=[]
for i in mutation_list_raw:
    if re.match(r'rs\d+',i.get('identifier')):
        dropout1.append(i)
    else:
        dropout2.append(i)

In [283]:
len(dropout1)

124

In [284]:
len(dropout2)

25

In [285]:
mutation2ncbi={}
for i in dropout1:
    mutation2ncbi[i.get('entity')]={}

In [286]:
len(mutation2ncbi)

69

In [287]:
mutation_converter=pickle.load(open('../ncbi/Mutation_converter.pkl','rb'))

In [288]:
len(mutation_converter)

1853

In [289]:
import time
import datetime
from multiprocessing import Process, Manager, Pool
import os, time, random
from lxml import etree
import requests
def get_mutation_info(i,final_mutations_inte):
    headers = {
        "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 \
        (KHTML, like Gecko) Chrome/83.0.4103.97 Safari/537.36"
    }
    url = 'https://www.ncbi.nlm.nih.gov/snp/' + i
    html = etree.HTML(requests.get(url=url,headers=headers).content)
    ncbisummary={
    'Position':''.join(html.xpath('//*[@class="usa-width-one-half"][1]/dd[2]/span[1]/text()'))+''.join(html.xpath('//*[@class="usa-width-one-half"][1]/dd[2]/span[2]/text()')),
    # 'Position':html.xpath('//*[@class="usa-width-one-half"][1]/dd[2]/span[1]/text()')+html.xpath('//*[@class="usa-width-one-half"][1]/dd[2]/span[2]/text()'),
    'Alleles':''.join(html.xpath('normalize-space(//*[@class="usa-width-one-half"][1]/dd[3]/text())')),
    'Clinical Significance':'https://www.ncbi.nlm.nih.gov'+''.join(html.xpath('//*[@class="usa-width-one-half"][2]/dd[1]/a/@href')) if html.xpath('//*[@class="usa-width-one-half"][2]/dd[1]/a/@href') else '',
    'Gene : Consequence':[''.join(html.xpath('//*[@class="usa-width-one-half"][2]/dd[2]/div[1]/text()')),''.join(html.xpath('//*[@class="usa-width-one-half"][2]/dd[2]/span[1]/text()'))] if ''.join(html.xpath('//*[@class="usa-width-one-half"][2]/dd[2]/div[1]/text()')) else [''.join(html.xpath('//*[@class="usa-width-one-half"][2]/dd[2]/span[1]/text()'))],
    'Genomic View':url+''.join(html.xpath('//*[@class="usa-width-one-half"][2]/dd[4]/a/@href')),
    }
    final_mutations_inte.update({i:ncbisummary})
    time.sleep(0.5)

In [290]:
judge=False
p = Pool(10)
manager=Manager()
final_mutations_inte = manager.dict()
for i in mutation2ncbi.keys():
    if not mutation_converter.get(i):
        p.apply_async(get_mutation_info, args=(i,final_mutations_inte))
        judge=True
p.close()
p.join()
if judge:
    mutation_converter.update(dict(final_mutations_inte))
    pickle.dump(mutation_converter, open('../ncbi/Mutation_converter.pkl','wb'))

In [291]:
len(mutation_converter)

1853

In [292]:
for i in range(len(mutation_list_raw)):
    if re.match(r'rs\d+',mutation_list_raw[i].get('entity')):
        mutation_list_raw[i]=dict(mutation_list_raw[i],**mutation_converter.get(mutation_list_raw[i].get('entity')))
    else:
        mutation_list_raw[i]=dict(mutation_list_raw[i],**{'Position':'','Alleles':'','Clinical Significance':'','Gene : Consequence':[''],'Genomic View':''})

In [293]:
import time
import datetime
import re
dropout=[]
months=['Jan','Feb','Mar','Apr','May','Jun','Jul','Aug','Sep','Oct','Nov','Dec']
import re

for i in mutation_list_raw:
    j=i.get('DP')
    if re.search('[S,s]upplement',j,flags=re.M|re.I):
        j=re.sub('[S,s]upplement.*','',j,flags=re.M|re.I)
    j=re.sub('\s?-\s?','-',j)
    j=re.sub('\.','',j)
    j=re.sub('Aout','Aug',j)
    if len(j.split(' '))==3:
        try:
            date=int(datetime.datetime.strftime(datetime.datetime.strptime(j,'%Y %b %d'), '%Y%m%d'))
        except:
            if j.split()[1] in months:
                if re.search(r'\d+',j.split()[2],flags=re.M|re.I):
                    j=j.split()[0]+' '+j.split()[1]+' '+ re.search(r'\d+',j.split()[2],flags=re.M|re.I).group()
                    try:
                        date=int(datetime.datetime.strftime(datetime.datetime.strptime(j,'%Y %b %d'), '%Y%m%d'))
                    except:
                        j=j[:4]
                        date=int(datetime.datetime.strftime(datetime.datetime.strptime(j,'%Y'), '%Y'+'0101'))
                        print('***************************************ERROR**********************************************')
                else:
                    j=j.split()[0]+' '+j.split()[1]+' '+ '1'
                    try:
                        date=int(datetime.datetime.strftime(datetime.datetime.strptime(j,'%Y %b %d'), '%Y%m%d'))
                    except:
                        print(j)
                        j=j[:4]
                        date=int(datetime.datetime.strftime(datetime.datetime.strptime(j,'%Y'), '%Y'+'0101'))
            elif j.split()[1] not in months:
                if re.search(r'([a-zA-Z]+)[-/]([a-zA-Z]+)',j.split()[1],flags=re.M|re.I) or re.search(r'([a-zA-Z]+)[-/](\d+)',j.split()[1],flags=re.M|re.I):
                    if re.search(r'([a-zA-Z]+)[-/]([a-zA-Z]+)',j.split()[1],flags=re.M|re.I):
                        middle=re.search(r'([a-zA-Z]+)[-/]([a-zA-Z]+)',j.split()[1],flags=re.M|re.I).group(1)
                    else:
                        middle=re.search(r'([a-zA-Z]+)[-/](\d+)',j.split()[1],flags=re.M|re.I).group(1)
                    if middle in months:
                        if re.search(r'\d+',j.split()[2],flags=re.M|re.I):
                            j=j.split()[0]+' '+middle+' '+ re.search(r'\d+',j.split()[2],flags=re.M|re.I).group()
                        else:
                            j=j.split()[0]+' '+middle+' '+ '1'
                    elif re.match(r'Sum(mer)?',middle):
                        if re.search(r'\d+',j.split()[2],flags=re.M|re.I):
                            j=j.split()[0]+' '+'Jun'+' '+ re.search(r'\d+',j.split()[2],flags=re.M|re.I).group()
                        else:
                            j=j.split()[0]+' '+'Jun'+' '+ '1'
                    elif re.match(r'Spr(ing)?',middle):
                        if re.search(r'\d+',j.split()[2],flags=re.M|re.I):
                            j=j.split()[0]+' '+'Mar'+' '+ re.search(r'\d+',j.split()[2],flags=re.M|re.I).group()
                        else:
                            j=j.split()[0]+' '+'Mar'+' '+ '1'
                    elif re.match(r'Win(ter)?',middle):
                        if re.search(r'\d+',j.split()[2],flags=re.M|re.I):
                            j=j.split()[0]+' '+'Dec'+' '+ re.search(r'\d+',j.split()[2],flags=re.M|re.I).group()
                        else:
                            j=j.split()[0]+' '+'Dec'+' '+ '1'    
                    elif re.match(r'Aut(umn)?',middle) or \
                    re.match(r'Fall',middle):
                        if re.search(r'\d+',j.split()[2],flags=re.M|re.I):
                            j=j.split()[0]+' '+'Sep'+' '+ re.search(r'\d+',j.split()[2],flags=re.M|re.I).group()
                        else:
                            j=j.split()[0]+' '+'Sep'+' '+ '1'      
                    try:
                        date=int(datetime.datetime.strftime(datetime.datetime.strptime(j,'%Y %b %d'), '%Y%m%d'))
                    except:
                        print(j)
                        j=j[:4]
                        date=int(datetime.datetime.strftime(datetime.datetime.strptime(j,'%Y'), '%Y'+'0101'))
                else:
                    j=j[:4]
                    date=int(datetime.datetime.strftime(datetime.datetime.strptime(j,'%Y'), '%Y'+'0101'))
    elif len(j.split())==2:
        if re.match(r'Sum(mer)?',j.split()[1]):
            date=int(j[:4]+'0601')
        elif re.match(r'Spr(ing)?',j.split()[1]):
            date=int(j[:4]+'0301')
        elif re.match(r'Win(ter)?',j.split()[1]):
            date=int(j[:4]+'1201')
        elif re.match(r'Aut(umn)?',j.split()[1]) or re.match(r'Fall',j.split()[1]):
            date=int(j[:4]+'0901')
        elif re.match(r'Sum(mer)?',j.split()[0]):
            date=int(j[-4:]+'0601')
        elif re.match(r'Spr(ing)?',j.split()[0]):
            date=int(j[-4:]+'0301')
        elif re.match(r'Win(ter)?',j.split()[0]):
            date=int(j[-4:]+'1201')
        elif re.match(r'Aut(umn)?',j.split()[0]) or re.match(r'Fall',j.split()[0]):
            date=int(j[-4:]+'0901')
        else:
            try:
                date=int(datetime.datetime.strftime(datetime.datetime.strptime(j,'%Y %b'), '%Y%m'+'01'))
            except:
                if re.search(r'([a-zA-Z]+)[-/]([a-zA-Z]+)',j.split()[1],flags=re.M|re.I) or re.search(r'([a-zA-Z]+)[-/](\d+)',j.split()[1],flags=re.M|re.I):
                    if re.search(r'([a-zA-Z]+)[-/]([a-zA-Z]+)',j.split()[1],flags=re.M|re.I):
                        middle=re.search(r'([a-zA-Z]+)[-/]([a-zA-Z]+)',j.split()[1],flags=re.M|re.I).group(1)
                    else:
                        middle=re.search(r'([a-zA-Z]+)[-/](\d+)',j.split()[1],flags=re.M|re.I).group(1)
                    if middle in months:
                        j=j.split()[0]+' '+middle+' '+ '1'
                    elif re.match(r'Sum(mer)?',middle):
                        j=j.split()[0]+' '+'Jun'+' '+ '1'
                    elif re.match(r'Spr(ing)?',middle):
                        j=j.split()[0]+' '+'Mar'+' '+ '1'
                    elif re.match(r'Win(ter)?',middle):
                        j=j.split()[0]+' '+'Dec'+' '+ '1'    
                    elif re.match(r'Aut(umn)?',middle) or \
                    re.match(r'Fall',middle):
                        j=j.split()[0]+' '+'Sep'+' '+ '1'      
                    try:
                        date=int(datetime.datetime.strftime(datetime.datetime.strptime(j,'%Y %b %d'), '%Y%m%d'))
                    except:
                        j=j[:4]
                        date=int(datetime.datetime.strftime(datetime.datetime.strptime(j,'%Y'), '%Y'+'0101'))
                elif re.search(r'(\d+)[-/](\d+)',j.split()[1],flags=re.M|re.I):
                    midd=re.search(r'(\d+)[-/](\d+)',j.split()[1],flags=re.M|re.I).group(1)
                    if len(midd)!=2:
                        midd='0'+midd
                    j=j[:4]
                    date=int(datetime.datetime.strftime(datetime.datetime.strptime(j,'%Y'), '%Y'+midd+'01'))
                else:
                    print(j)
                    j=j[:4]
                    date=int(datetime.datetime.strftime(datetime.datetime.strptime(j,'%Y'), '%Y'+'0101'))
    elif len(j.split())==1:
        j=j[:4]
        date=int(datetime.datetime.strftime(datetime.datetime.strptime(j,'%Y'), '%Y'+'0101'))
    else:
        j=j.split()[0]+' '+j.split()[1]+' '+re.search(r'\d+',j.split()[2],flags=re.M|re.I).group()
        date=int(datetime.datetime.strftime(datetime.datetime.strptime(j,'%Y %b %d'), '%Y%m%d'))
    i['date']=date
    i['year']=int(str(i.get('date'))[:4])

In [294]:
len(mutation_list_raw)

149

In [295]:
mutation_list_raw[0]

{'AB': 'Cognitive decline is part of the normal aging process. However, some people experience a more rapid decline than others due to environmental and genetic factors. Numerous single nucleotide polymorphisms (SNPs) have been linked to cognitive function, but only a few to cognitive decline. To understand whether cognitive function and cognitive decline are driven by the same mechanisms, we investigated whether 433 SNPs previously linked to cognitive function and 2 SNPs previously linked to cognitive decline are associated with both general cognitive functioning at baseline and general cognitive decline up to 20-years follow-up in the Doetinchem Cohort Study (DCS). The DCS is a longitudinal population-based study that enrolled men and women aged 20-59 years between 1987-1991, with follow-up examinations every 5 years. We used data of rounds 2-6 (1993-2017, n = 2559). General cognitive function was assessed using four cognition tests measuring memory, speed, fluency and flexibility. W

In [296]:
Mutation_raw=copy.deepcopy(mutation_list_raw)

In [297]:
collections_Mutation_raw.insert_many(Mutation_raw)

# Mutation_all

In [298]:
collections_Mutation_all=db.Mutation_all

In [483]:
Mutation_testcopy=list(collections_Mutation_raw.find())

In [484]:
len(Mutation_testcopy)

6752

In [485]:
for i in Mutation_testcopy:
    i['type']='Mutation'
    i["PMID"]=[i.get("PMID")]
    i["sentence"]=[[i.get("sentence")]]
    pmid=i.get('PMID')[0]
    i['DP']=[i.get("DP")]
    i['AB']=[i.get("AB")]
    i['URL']='https://pubmed.ncbi.nlm.nih.gov/'+pmid
    i['TI']=[i.get("TI")]
    i['target']=[[i.get('word')]]
    i["date"]=[i.get("date")]
    i["year"]=[i.get("year")]
    i['JT']=[i.get('JT')]

In [486]:
Mutation_mid={}
for i in Mutation_testcopy:
    if not Mutation_mid.get(i.get("entity")):
        Mutation_mid[i.get("entity")]=i
    else:
        if i.get('PMID')[0] in Mutation_mid.get(i.get('entity')).get('PMID'):

            ind=Mutation_mid.get(i.get('entity')).get('PMID').index(i.get('PMID')[0])
            if i.get("sentence")[0][0] in Mutation_mid.get(i.get('entity')).get('sentence')[ind]:
                continue
            else:
                Mutation_mid[i.get('entity')]['sentence'][ind] = Mutation_mid.get(i.get('entity')).get('sentence')[ind] + i.get("sentence")[0]
                Mutation_mid[i.get('entity')]['target'][ind] = Mutation_mid.get(i.get('entity')).get('target')[ind] + i.get("target")[0]

        else:
            Mutation_mid[i.get("entity")]["PMID"]=Mutation_mid.get(i.get("entity")).get("PMID")+i.get("PMID")
            Mutation_mid[i.get("entity")]["sentence"]=Mutation_mid.get(i.get("entity")).get("sentence")+i.get("sentence")
            Mutation_mid[i.get("entity")]["AB"]=Mutation_mid.get(i.get("entity")).get("AB")+i.get("AB")
            Mutation_mid[i.get("entity")]["DP"]=Mutation_mid.get(i.get("entity")).get("DP")+i.get("DP")
            Mutation_mid[i.get("entity")]["TI"]=Mutation_mid.get(i.get("entity")).get("TI")+i.get("TI")
            Mutation_mid[i.get("entity")]["target"]=Mutation_mid.get(i.get("entity")).get("target")+i.get("target")
            Mutation_mid[i.get("entity")]["date"]=Mutation_mid.get(i.get("entity")).get("date")+i.get("date")
            Mutation_mid[i.get("entity")]["year"]=Mutation_mid.get(i.get("entity")).get("year")+i.get("year")
            Mutation_mid[i.get("entity")]["JT"]=Mutation_mid.get(i.get("entity")).get("JT")+i.get("JT")

In [487]:
Mutation_all=list(Mutation_mid.values())

In [488]:
for i in Mutation_all:
    i['allyears']=[]
    i['datestring']=[]
    for j in i.get('date'):
        if int(str(j)[:4]) not in i.get('allyears'):
            i['allyears']= i.get('allyears')+[int(str(j)[:4])]
    for j in i.get('date'):
        if str(j)[:4]+"-"+str(j)[4:6]+"-"+str(j)[6:] not in i.get('datestring'):
            i['datestring']=i.get('datestring')+[str(j)[:4]+"-"+str(j)[4:6]+"-"+str(j)[6:]]
    i['allyears']=sorted(i.get('allyears'), reverse=True)
    i['datestring']=sorted(i.get('datestring'), reverse=True)
    upperlist=set()
    for j in sum(i.get('target'),[]):
        upperlist.add(j.upper())
    i['targetshapes']=list(upperlist)
    i['addtime']=now_time

In [489]:
len(Mutation_all)

2612

# Disease_raw

In [162]:
collections_Disease_raw=db.Disease_raw

collections_Disease_raw.remove({'addtime':now_time})

In [307]:
Disease_pubtator_addpmid={}
for i in pubtator_raw:
    if i.get('type')=='Disease':
        if Disease_pubtator_addpmid.get(i.get('id')):
            Disease_pubtator_addpmid[i.get('id')]=Disease_pubtator_addpmid.get(i.get('id'))+[i]
        else:
            Disease_pubtator_addpmid[i.get('id')]=[i]

In [308]:
Disease_output_json=[]
for i,j in Disease_pubtator_addpmid.items():
    Disease_output_json+=j

In [309]:
len(Disease_output_json)

29495

In [310]:
Disease_mesh=pickle.load(open('../disease/Disease.pkl','rb'))

In [311]:
Disease_mesh=Disease_mesh[1:]

In [312]:
for i in Disease_mesh:
    if i.get('text')=='Uveitis, Suppurative':
        i['MeSHUniqueID']='D015829'

In [551]:
for i in Disease_mesh:
    if i.get('text')=='Hearing Loss':
        print(i)

{'text': 'Hearing Loss', 'url': '/mesh/68034381', 'MeSHUniqueID': 'D034381', 'type': 'Disease'}
{'text': 'Hearing Loss', 'url': '/mesh/68034381', 'MeSHUniqueID': 'D034381', 'type': 'Disease'}
{'text': 'Hearing Loss', 'url': '/mesh/68034381', 'MeSHUniqueID': 'D034381', 'type': 'Disease'}


In [313]:
diseaselist=[]
notdiseaselist=[]
for i in Disease_mesh:
    if i.get('MeSHUniqueID'):
        diseaselist.append({'MESH':'MESH:'+i.get('MeSHUniqueID'),'type':i.get('type'),'url':i.get('url'),'text':i.get('text')})
    else:
        notdiseaselist.append(i)

In [314]:
len(diseaselist)

12791

In [643]:
len(notdiseaselist)

0

In [315]:
disease2classdict={}
count=0
for i in diseaselist:
    if not disease2classdict.get(i.get('MESH')):
        disease2classdict[i.get('MESH')]=[i.get('type')]
    else:
        if i.get('type') in disease2classdict.get(i.get('MESH')):
            pass
        else:
            disease2classdict[i.get('MESH')]=disease2classdict.get(i.get('MESH'))+[i.get('type')]
            count+=1
            if count<=5:
                print(i.get('MESH')+'\t'+str(disease2classdict.get(i.get('MESH'))))

In [316]:
disease2subclassdict={}
count=0
for i in diseaselist:
    disease2subclassdict[i.get('MESH')]=i.get('text')

In [317]:
disease2urldict={}
count=0
for i in diseaselist:
    disease2urldict[i.get('MESH')]=i.get('url')

In [318]:
len(disease2classdict)

4915

In [319]:
for i in Disease_output_json:
    i['PMID']=i.get('id')
    i['word']=i.get('text')
    del i['id']
    del i['text']

In [320]:
Disease_final=[]
Disease_dropout=[]

In [321]:
for i in Disease_output_json:
    if i.get('identifier') in disease2classdict.keys():
        Disease_final.append(dict(i,**{'class':disease2classdict.get(i.get('identifier')),'subclass':disease2subclassdict.get(i.get('identifier')),'url':disease2urldict.get(i.get('identifier'))}))
    else:
        Disease_dropout.append(i)

In [699]:
len(Disease_output_json)

29495

In [698]:
len(Disease_dropout)

6248

In [555]:
Disease_deduplicated=set()
Disease_deduplicated_count=0
disease_list_raw=[]
for i in Disease_final:
    k=i.get('PMID')
    i['DP']=rawpickwithAB.get(k).get('DP')
    i['AB']=rawpickwithAB.get(k).get('AB')
    i['URL']='https://pubmed.ncbi.nlm.nih.gov/'+k
    i['TI']=rawpickwithAB.get(k).get('TI')
    i['entity']=i.get('subclass')
    i['target']=i.get('word')
    i['type']='Disease'
    i['JT']=rawpickwithAB.get(k).get('JT')
    if i.get('identifier') and re.match(r'MESH:',i.get('identifier')):
        i['MESH']=i.get('identifier')[5:]
    else:
        i['MESH']=''
    # tuples=(i.get("PMID"),i.get("entity"),i.get("word"),i.get("sentence"))
    tuples=(i.get("PMID"),i.get("entity"),i.get("word"),i.get("sentence"))
    if tuples not in Disease_deduplicated:
        Disease_deduplicated.add(tuples)
        disease_list_raw.append(i)
    else:
        Disease_deduplicated_count+=1
        continue

In [556]:
Disease_deduplicated_count

1418

In [557]:
disease2ncbi={}
for i in disease_list_raw:
    disease2ncbi[i.get('url')]=''

In [558]:
len(disease2ncbi)

1113

In [559]:
disease_converter=pickle.load(open('../ncbi/Disease_converter.pkl','rb'))

In [560]:
len(disease_converter)

3344

In [561]:
import time
import datetime
from multiprocessing import Process, Manager, Pool
import os, time, random
from lxml import etree
import requests
def get_disease_info(i,final_diseases_inte):
    headers = {
        "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 \
        (KHTML, like Gecko) Chrome/83.0.4103.97 Safari/537.36"
    }
    url = 'https://www.ncbi.nlm.nih.gov' + i
    
    res=requests.get(url=url,headers=headers)
    html = etree.HTML(res.content)
    ncbisummary={
    # 'numbersOfLiterature' : len(i.get('PMID')),
    'Summary' : ''.join(html.xpath('//*[@class="mesh_ds_scope_note"]/text()'))
    }
    final_diseases_inte.update({i:ncbisummary})
    time.sleep(0.5)

In [562]:
judge=False
p = Pool(10)
manager=Manager()
final_diseases_inte = manager.dict()
for i in disease2ncbi.keys():
    if not disease_converter.get(i):
        p.apply_async(get_disease_info, args=(i,final_diseases_inte))
        judge=True
p.close()
p.join()
if judge:
    disease_converter.update(dict(final_diseases_inte))
    pickle.dump(disease_converter, open('../ncbi/Disease_converter.pkl','wb'))

In [563]:
len(disease_converter)

3344

In [564]:
for i in range(len(disease_list_raw)):
    disease_list_raw[i]=dict(disease_list_raw[i],**disease_converter.get(disease_list_raw[i].get('url')))

In [565]:
Disease_deduplicated_dropout=set()
Disease_deduplicated_dropout_count=0
Disease_dropout_raw=[]
for i in Disease_dropout:
    k=i.get('PMID')
    i['DP']=rawpickwithAB.get(k).get('DP')
    i['AB']=rawpickwithAB.get(k).get('AB')
    i['URL']='https://pubmed.ncbi.nlm.nih.gov/'+k
    i['TI']=rawpickwithAB.get(k).get('TI')
    i['entity']=i.get('word')
    i['target']=i.get('word')
    i['type']='Disease'
    i['JT']=rawpickwithAB.get(k).get('JT')
    if i.get('identifier') and re.match('MESH:',i.get('identifier')):
        i['MESH']=i.get('identifier')[5:]
    else:
        i['MESH']=''
    # tuples=(i.get("PMID"),i.get("entity"),i.get("word"),i.get("sentence"))
    tuples=(i.get("PMID"),i.get("entity"),i.get("word"),i.get("sentence"))
    if tuples not in Disease_deduplicated_dropout:
        Disease_deduplicated_dropout.add(tuples)
        Disease_dropout_raw.append(i)
    else:
        Disease_deduplicated_dropout_count+=1
        continue

In [566]:
for i in Disease_dropout_raw:
    i['Summary']=''

In [697]:
len(Disease_dropout_raw)

5901

In [567]:
Disease_raw=disease_list_raw+Disease_dropout_raw

In [568]:
len(Disease_raw)

27730

In [569]:
import time
import datetime
import re
dropout=[]
months=['Jan','Feb','Mar','Apr','May','Jun','Jul','Aug','Sep','Oct','Nov','Dec']
import re

for i in Disease_raw:
    j=i.get('DP')
    if re.search('[S,s]upplement',j,flags=re.M|re.I):
        j=re.sub('[S,s]upplement.*','',j,flags=re.M|re.I)
    j=re.sub('\s?-\s?','-',j)
    j=re.sub('\.','',j)
    j=re.sub('Aout','Aug',j)
    if len(j.split(' '))==3:
        try:
            date=int(datetime.datetime.strftime(datetime.datetime.strptime(j,'%Y %b %d'), '%Y%m%d'))
        except:
            if j.split()[1] in months:
                if re.search(r'\d+',j.split()[2],flags=re.M|re.I):
                    j=j.split()[0]+' '+j.split()[1]+' '+ re.search(r'\d+',j.split()[2],flags=re.M|re.I).group()
                    try:
                        date=int(datetime.datetime.strftime(datetime.datetime.strptime(j,'%Y %b %d'), '%Y%m%d'))
                    except:
                        j=j[:4]
                        date=int(datetime.datetime.strftime(datetime.datetime.strptime(j,'%Y'), '%Y'+'0101'))
                        print('***************************************ERROR**********************************************')
                else:
                    j=j.split()[0]+' '+j.split()[1]+' '+ '1'
                    try:
                        date=int(datetime.datetime.strftime(datetime.datetime.strptime(j,'%Y %b %d'), '%Y%m%d'))
                    except:
                        print(j)
                        j=j[:4]
                        date=int(datetime.datetime.strftime(datetime.datetime.strptime(j,'%Y'), '%Y'+'0101'))
            elif j.split()[1] not in months:
                if re.search(r'([a-zA-Z]+)[-/]([a-zA-Z]+)',j.split()[1],flags=re.M|re.I) or re.search(r'([a-zA-Z]+)[-/](\d+)',j.split()[1],flags=re.M|re.I):
                    if re.search(r'([a-zA-Z]+)[-/]([a-zA-Z]+)',j.split()[1],flags=re.M|re.I):
                        middle=re.search(r'([a-zA-Z]+)[-/]([a-zA-Z]+)',j.split()[1],flags=re.M|re.I).group(1)
                    else:
                        middle=re.search(r'([a-zA-Z]+)[-/](\d+)',j.split()[1],flags=re.M|re.I).group(1)
                    if middle in months:
                        if re.search(r'\d+',j.split()[2],flags=re.M|re.I):
                            j=j.split()[0]+' '+middle+' '+ re.search(r'\d+',j.split()[2],flags=re.M|re.I).group()
                        else:
                            j=j.split()[0]+' '+middle+' '+ '1'
                    elif re.match(r'Sum(mer)?',middle):
                        if re.search(r'\d+',j.split()[2],flags=re.M|re.I):
                            j=j.split()[0]+' '+'Jun'+' '+ re.search(r'\d+',j.split()[2],flags=re.M|re.I).group()
                        else:
                            j=j.split()[0]+' '+'Jun'+' '+ '1'
                    elif re.match(r'Spr(ing)?',middle):
                        if re.search(r'\d+',j.split()[2],flags=re.M|re.I):
                            j=j.split()[0]+' '+'Mar'+' '+ re.search(r'\d+',j.split()[2],flags=re.M|re.I).group()
                        else:
                            j=j.split()[0]+' '+'Mar'+' '+ '1'
                    elif re.match(r'Win(ter)?',middle):
                        if re.search(r'\d+',j.split()[2],flags=re.M|re.I):
                            j=j.split()[0]+' '+'Dec'+' '+ re.search(r'\d+',j.split()[2],flags=re.M|re.I).group()
                        else:
                            j=j.split()[0]+' '+'Dec'+' '+ '1'    
                    elif re.match(r'Aut(umn)?',middle) or \
                    re.match(r'Fall',middle):
                        if re.search(r'\d+',j.split()[2],flags=re.M|re.I):
                            j=j.split()[0]+' '+'Sep'+' '+ re.search(r'\d+',j.split()[2],flags=re.M|re.I).group()
                        else:
                            j=j.split()[0]+' '+'Sep'+' '+ '1'      
                    try:
                        date=int(datetime.datetime.strftime(datetime.datetime.strptime(j,'%Y %b %d'), '%Y%m%d'))
                    except:
                        print(j)
                        j=j[:4]
                        date=int(datetime.datetime.strftime(datetime.datetime.strptime(j,'%Y'), '%Y'+'0101'))
                else:
                    j=j[:4]
                    date=int(datetime.datetime.strftime(datetime.datetime.strptime(j,'%Y'), '%Y'+'0101'))
    elif len(j.split())==2:
        if re.match(r'Sum(mer)?',j.split()[1]):
            date=int(j[:4]+'0601')
        elif re.match(r'Spr(ing)?',j.split()[1]):
            date=int(j[:4]+'0301')
        elif re.match(r'Win(ter)?',j.split()[1]):
            date=int(j[:4]+'1201')
        elif re.match(r'Aut(umn)?',j.split()[1]) or re.match(r'Fall',j.split()[1]):
            date=int(j[:4]+'0901')
        elif re.match(r'Sum(mer)?',j.split()[0]):
            date=int(j[-4:]+'0601')
        elif re.match(r'Spr(ing)?',j.split()[0]):
            date=int(j[-4:]+'0301')
        elif re.match(r'Win(ter)?',j.split()[0]):
            date=int(j[-4:]+'1201')
        elif re.match(r'Aut(umn)?',j.split()[0]) or re.match(r'Fall',j.split()[0]):
            date=int(j[-4:]+'0901')
        else:
            try:
                date=int(datetime.datetime.strftime(datetime.datetime.strptime(j,'%Y %b'), '%Y%m'+'01'))
            except:
                if re.search(r'([a-zA-Z]+)[-/]([a-zA-Z]+)',j.split()[1],flags=re.M|re.I) or re.search(r'([a-zA-Z]+)[-/](\d+)',j.split()[1],flags=re.M|re.I):
                    if re.search(r'([a-zA-Z]+)[-/]([a-zA-Z]+)',j.split()[1],flags=re.M|re.I):
                        middle=re.search(r'([a-zA-Z]+)[-/]([a-zA-Z]+)',j.split()[1],flags=re.M|re.I).group(1)
                    else:
                        middle=re.search(r'([a-zA-Z]+)[-/](\d+)',j.split()[1],flags=re.M|re.I).group(1)
                    if middle in months:
                        j=j.split()[0]+' '+middle+' '+ '1'
                    elif re.match(r'Sum(mer)?',middle):
                        j=j.split()[0]+' '+'Jun'+' '+ '1'
                    elif re.match(r'Spr(ing)?',middle):
                        j=j.split()[0]+' '+'Mar'+' '+ '1'
                    elif re.match(r'Win(ter)?',middle):
                        j=j.split()[0]+' '+'Dec'+' '+ '1'    
                    elif re.match(r'Aut(umn)?',middle) or \
                    re.match(r'Fall',middle):
                        j=j.split()[0]+' '+'Sep'+' '+ '1'      
                    try:
                        date=int(datetime.datetime.strftime(datetime.datetime.strptime(j,'%Y %b %d'), '%Y%m%d'))
                    except:
                        j=j[:4]
                        date=int(datetime.datetime.strftime(datetime.datetime.strptime(j,'%Y'), '%Y'+'0101'))
                elif re.search(r'(\d+)[-/](\d+)',j.split()[1],flags=re.M|re.I):
                    midd=re.search(r'(\d+)[-/](\d+)',j.split()[1],flags=re.M|re.I).group(1)
                    if len(midd)!=2:
                        midd='0'+midd
                    j=j[:4]
                    date=int(datetime.datetime.strftime(datetime.datetime.strptime(j,'%Y'), '%Y'+midd+'01'))
                else:
                    print(j)
                    j=j[:4]
                    date=int(datetime.datetime.strftime(datetime.datetime.strptime(j,'%Y'), '%Y'+'0101'))
    elif len(j.split())==1:
        j=j[:4]
        date=int(datetime.datetime.strftime(datetime.datetime.strptime(j,'%Y'), '%Y'+'0101'))
    else:
        j=j.split()[0]+' '+j.split()[1]+' '+re.search(r'\d+',j.split()[2],flags=re.M|re.I).group()
        date=int(datetime.datetime.strftime(datetime.datetime.strptime(j,'%Y %b %d'), '%Y%m%d'))
    i['date']=date
    i['year']=int(str(i.get('date'))[:4])

In [573]:
collections_Disease_raw.insert_many(Disease_raw)

In [645]:

qq=0
for i in Disease_raw:
    if not i.get('Summary'):
        qq+=1

In [646]:
qq

5901

In [647]:
len(Disease_raw)

27730

In [644]:
for i in Disease_raw:
    if not i.get('Summary'):
        print(i)
        break
    # if i.get('identifier') and not i.get('MESH'):
    #     if i.get('identifier')[:4]=='MESH':
    #         print(i)
    #         break

{'AB': "The title usage of Unde venisti 'from where have you come' is from a now dead language (Latin) that foundationally influenced modern English (not the major influence, but an essential formative one). This is an apt analogy for how both the ancient eukaryotic and eumetazoan functions of PGRMC proteins (PGRMC1 and PGRMC2 in mammals) probably influence modern human biology: via a formative trajectory from an evolutionarily foundational fulcrum. There is an arguable probability, although not a certainty, that PGRMC-like proteins were involved in eukaryogenesis. If so, then the proto-eukaryotic ancestral protein is modelled as having initiated the oxygen-induced and CYP450 (Cytochrome P450)-mediated synthesis of sterols in the endoplasmic reticulum to regulate proto-mitochondrial activity and heme homeostasis, as well as having enabled sterol transport between endoplasmic reticulum (ER) and mitochondria membranes involving the actin cytoskeleton, transport of heme from mitochondria,

disease_all_today=list(collections_Disease_raw.find())

for i in disease_all_today:
    if i.get('identifier') and re.match('MESH:',i.get('identifier')):
        i['MESH']=i.get('identifier')[5:]
    else:
        i['MESH']=''

i

for i in disease_all_today:
    if i.get('identifier') and not i.get('MESH'):
        if i.get('identifier')[:4]=='MESH':
            print(i)
            break

In [ ]:
collections_Disease_raw.insert_many(disease_all_today)

In [ ]:
collections_Disease_raw.remove({'addtime':now_time})

Disease_raw[0]

# Disease_all

In [607]:
collections_Disease_all=db.Disease_all

In [163]:
Disease_raw_list=list(collections_Disease_raw.find())

In [164]:
for i in disease_all_today:
    if i.get('Summary') and not i.get('Mesh'):
        print(i.get('Summary'))
        print(i.get('entity'))
        print(i.get('addtime'))
        print(i.get('identifier'))
        print(i.get('Mesh'))
        break

NameError: name 'disease_all_today' is not defined

In [165]:
for i in Disease_raw_list:
    i["PMID"]=[i.get("PMID")]
    i["sentence"]=[[i.get("sentence")]]
    pmid=i.get('PMID')[0]
    i['DP']=[i.get("DP")]
    i['AB']=[i.get("AB")]
    i['URL']='https://pubmed.ncbi.nlm.nih.gov/'+pmid
    i['TI']=[i.get("TI")]
    i['target']=[[i.get('word')]]
    i["date"]=[i.get("date")]
    i["year"]=[i.get("year")]
    i['JT']=[i.get('JT')]

In [166]:
i

{'_id': ObjectId('63fa4073648b16758fcc08f4'),
 'AB': ['SIGNIFICANCE: Oxygen is indispensable for aerobic life but its utilization exposes cells and tissues to oxidative stress; thus, tight regulation of cellular, tissue and systemic oxygen concentrations is crucial. Here, we review the current understanding of how the human organism (mal-)adapts to low (hypoxia) and high (hyperoxia) oxygen levels and how these adaptations may be harnessed as therapeutic or performance enhancing strategies at the systemic level. Recent Advances: Hyperbaric oxygen therapy is already a cornerstone of modern medicine and the application of mild hypoxia, i.e., hypoxia conditioning, to strengthen the resilience of organs or the whole body to severe hypoxic insults is important preparation for high-altitude sojourns or to protect the cardiovascular system from hypoxic/ischemic damage. Many other applications of adaptations to hypo- and/or hyperoxia are only just emerging. Hypoxia conditioning - sometimes in c

In [167]:
Disease_mid={}
for i in Disease_raw_list:
    if not Disease_mid.get(i.get("entity")):
        Disease_mid[i.get("entity")]=i
    else:
        if i.get('PMID')[0] in Disease_mid.get(i.get('entity')).get('PMID'):

            ind=Disease_mid.get(i.get('entity')).get('PMID').index(i.get('PMID')[0])
            if i.get("sentence")[0][0] in Disease_mid.get(i.get('entity')).get('sentence')[ind]:
                continue
            else:
                Disease_mid[i.get('entity')]['sentence'][ind] = Disease_mid.get(i.get('entity')).get('sentence')[ind] + i.get("sentence")[0]
                Disease_mid[i.get('entity')]['target'][ind] = Disease_mid.get(i.get('entity')).get('target')[ind] + i.get("target")[0]

        else:
            Disease_mid[i.get("entity")]["PMID"]=Disease_mid.get(i.get("entity")).get("PMID")+i.get("PMID")
            Disease_mid[i.get("entity")]["sentence"]=Disease_mid.get(i.get("entity")).get("sentence")+i.get("sentence")
            Disease_mid[i.get("entity")]["AB"]=Disease_mid.get(i.get("entity")).get("AB")+i.get("AB")
            Disease_mid[i.get("entity")]["DP"]=Disease_mid.get(i.get("entity")).get("DP")+i.get("DP")
            Disease_mid[i.get("entity")]["TI"]=Disease_mid.get(i.get("entity")).get("TI")+i.get("TI")
            Disease_mid[i.get("entity")]["target"]=Disease_mid.get(i.get("entity")).get("target")+i.get("target")
            Disease_mid[i.get("entity")]["date"]=Disease_mid.get(i.get("entity")).get("date")+i.get("date")
            Disease_mid[i.get("entity")]["year"]=Disease_mid.get(i.get("entity")).get("year")+i.get("year")
            Disease_mid[i.get("entity")]["JT"]=Disease_mid.get(i.get("entity")).get("JT")+i.get("JT")

In [168]:
Disease_output_json_final=list(Disease_mid.values())

In [169]:
# 17330
len(Disease_output_json_final)

17330

In [170]:
for i in Disease_output_json_final:
    i['allyears']=[]
    i['datestring']=[]
    for j in i.get('date'):
        if int(str(j)[:4]) not in i.get('allyears'):
            i['allyears']= i.get('allyears')+[int(str(j)[:4])]
    for j in i.get('date'):
        if str(j)[:4]+"-"+str(j)[4:6]+"-"+str(j)[6:] not in i.get('datestring'):
            i['datestring']=i.get('datestring')+[str(j)[:4]+"-"+str(j)[4:6]+"-"+str(j)[6:]]
    i['allyears']=sorted(i.get('allyears'), reverse=True)
    i['datestring']=sorted(i.get('datestring'), reverse=True)
    upperlist=set()
    for j in sum(i.get('target'),[]):
        upperlist.add(j.upper())
    i['targetshapes']=list(upperlist)
    i['addtime']=now_time

In [171]:
Disease_all=copy.deepcopy(Disease_output_json_final)

In [172]:
len(Disease_all)

17330

In [490]:
pwd

'/root/hald_autoupdate/2022_12_09'

In [510]:
names=locals()

In [610]:
entity_pairs=['Gene','RNA','Carbohydrate','Lipid','Peptide', 'Pharmaceutical_Preparations', 'Protein', 'Toxin','Mutation', 'Disease']

for i in entity_pairs:
    for j in names[i+'_all']:
        j['addtime']=now_time

In [611]:
for i in entity_pairs:
    pickle.dump(names[i+'_all'],open('results/'+i+'_all.pkl','wb'))

In [513]:
from nltk.tokenize import sent_tokenize

In [559]:
for i in Gene_all:
    if i.get('Also_known_as')=='''The protein encoded by this gene is a cofactor needed by pancreatic lipase for efficient dietary lipid hydrolysis. It binds to the C-terminal, non-catalytic domain of lipase, thereby stabilizing an active conformation and considerably increasing the overall hydrophobic binding site. The gene product allows lipase to anchor noncovalently to the surface of lipid micelles, counteracting the destabilizing influence of intestinal bile salts. This cofactor is only expressed in pancreatic acinar cells, suggesting regulation of expression by tissue-specific elements. Three transcript variants encoding different isoforms have been found for this gene. [provided by RefSeq, Nov 2011]''':
        print(i.get('entity'))
        break

CLPS


for i in Gene_raw_list:
    if i.get('entity')=='FZD1':
        print(i)
        break

In [612]:
all_es=[]
mu=0
di=0
noturl=[]
for i in Gene_all:
    if i.get('Summary'):
        all_es.append({'entity':i.get('entity'),'PMID':i.get('PMID'),'literature':len(i.get('PMID')),'targetshapes':i.get('targetshapes'),'allyears':i.get('allyears'),'type':i.get('type'),'Also_known_as':i.get('Also_known_as'),'Summary':sent_tokenize(i.get('Summary'))[0],'_id':i.get('type')+'_'+i.get('entity'),'datest':int(i.get('datestring')[0].replace('-',''))})
    else:
        all_es.append({'entity':i.get('entity'),'PMID':i.get('PMID'),'literature':len(i.get('PMID')),'targetshapes':i.get('targetshapes'),'allyears':i.get('allyears'),'type':i.get('type'),'Also_known_as':i.get('Also_known_as'),'Summary':'','_id':i.get('type')+'_'+i.get('entity'),'datest':int(i.get('datestring')[0].replace('-',''))})
for i in RNA_all:
    if i.get('Summary'):
        all_es.append({'entity':i.get('entity'),'PMID':i.get('PMID'),'literature':len(i.get('PMID')),'targetshapes':i.get('targetshapes'),'allyears':i.get('allyears'),'type':i.get('type'),'specific_type':i.get('specific_type'),'Also_known_as':i.get('Also_known_as'),'Summary':sent_tokenize(i.get('Summary'))[0],'_id':i.get('type')+'_'+i.get('entity'),'datest':int(i.get('datestring')[0].replace('-',''))})
    else:
        all_es.append({'entity':i.get('entity'),'PMID':i.get('PMID'),'literature':len(i.get('PMID')),'targetshapes':i.get('targetshapes'),'allyears':i.get('allyears'),'type':i.get('type'),'specific_type':i.get('specific_type'),'Also_known_as':i.get('Also_known_as'),'Summary':'','_id':i.get('type')+'_'+i.get('entity'),'datest':int(i.get('datestring')[0].replace('-',''))})
for i in Carbohydrate_all:
    if i.get('Summary'):
        all_es.append({'entity':i.get('entity'),'PMID':i.get('PMID'),'literature':len(i.get('PMID')),'targetshapes':i.get('targetshapes'),'allyears':i.get('allyears'),'type':i.get('type'),'MeshID':i.get('MESH'),'Summary':sent_tokenize(i.get('Summary'))[0],'url':'https://www.ncbi.nlm.nih.gov'+i.get('url'),'_id':i.get('type')+'_'+i.get('entity'),'datest':int(i.get('datestring')[0].replace('-',''))})
    else:
        all_es.append({'entity':i.get('entity'),'PMID':i.get('PMID'),'literature':len(i.get('PMID')),'targetshapes':i.get('targetshapes'),'allyears':i.get('allyears'),'type':i.get('type'),'MeshID':i.get('MESH'),'Summary':'','url':'https://www.ncbi.nlm.nih.gov'+i.get('url'),'_id':i.get('type')+'_'+i.get('entity'),'datest':int(i.get('datestring')[0].replace('-',''))})
for i in Lipid_all:
    if i.get('Summary'):
        all_es.append({'entity':i.get('entity'),'PMID':i.get('PMID'),'literature':len(i.get('PMID')),'targetshapes':i.get('targetshapes'),'allyears':i.get('allyears'),'type':i.get('type'),'MeshID':i.get('MESH'),'Summary':sent_tokenize(i.get('Summary'))[0],'url':'https://www.ncbi.nlm.nih.gov'+i.get('url'),'_id':i.get('type')+'_'+i.get('entity'),'datest':int(i.get('datestring')[0].replace('-',''))})
    else:
        all_es.append({'entity':i.get('entity'),'PMID':i.get('PMID'),'literature':len(i.get('PMID')),'targetshapes':i.get('targetshapes'),'allyears':i.get('allyears'),'type':i.get('type'),'MeshID':i.get('MESH'),'Summary':'','url':'https://www.ncbi.nlm.nih.gov'+i.get('url'),'_id':i.get('type')+'_'+i.get('entity'),'datest':int(i.get('datestring')[0].replace('-',''))})
for i in Peptide_all:
    if i.get('Summary'):
        all_es.append({'entity':i.get('entity'),'PMID':i.get('PMID'),'literature':len(i.get('PMID')),'targetshapes':i.get('targetshapes'),'allyears':i.get('allyears'),'type':i.get('type'),'MeshID':i.get('MESH'),'Summary':sent_tokenize(i.get('Summary'))[0],'url':'https://www.ncbi.nlm.nih.gov'+i.get('url'),'_id':i.get('type')+'_'+i.get('entity'),'datest':int(i.get('datestring')[0].replace('-',''))})
    else:
        all_es.append({'entity':i.get('entity'),'PMID':i.get('PMID'),'literature':len(i.get('PMID')),'targetshapes':i.get('targetshapes'),'allyears':i.get('allyears'),'type':i.get('type'),'MeshID':i.get('MESH'),'Summary':'','url':'https://www.ncbi.nlm.nih.gov'+i.get('url'),'_id':i.get('type')+'_'+i.get('entity'),'datest':int(i.get('datestring')[0].replace('-',''))})
for i in Pharmaceutical_Preparations_all:
    if i.get('Summary'):
        all_es.append({'entity':i.get('entity'),'PMID':i.get('PMID'),'literature':len(i.get('PMID')),'targetshapes':i.get('targetshapes'),'allyears':i.get('allyears'),'type':i.get('type'),'MeshID':i.get('MESH'),'Summary':sent_tokenize(i.get('Summary'))[0],'url':'https://www.ncbi.nlm.nih.gov'+i.get('url'),'_id':'PharmaceuticalPreparations'+'_'+i.get('entity'),'datest':int(i.get('datestring')[0].replace('-',''))})
    else:
        all_es.append({'entity':i.get('entity'),'PMID':i.get('PMID'),'literature':len(i.get('PMID')),'targetshapes':i.get('targetshapes'),'allyears':i.get('allyears'),'type':i.get('type'),'MeshID':i.get('MESH'),'Summary':'','url':'https://www.ncbi.nlm.nih.gov'+i.get('url'),'_id':'PharmaceuticalPreparations'+'_'+i.get('entity'),'datest':int(i.get('datestring')[0].replace('-',''))})
for i in Protein_all:
    if i.get('Summary'):
        all_es.append({'entity':i.get('entity'),'PMID':i.get('PMID'),'literature':len(i.get('PMID')),'targetshapes':i.get('targetshapes'),'allyears':i.get('allyears'),'type':i.get('type'),'MeshID':i.get('MESH'),'Summary':sent_tokenize(i.get('Summary'))[0],'url':'https://www.ncbi.nlm.nih.gov'+i.get('url'),'_id':i.get('type')+'_'+i.get('entity'),'datest':int(i.get('datestring')[0].replace('-',''))})
    else:
        all_es.append({'entity':i.get('entity'),'PMID':i.get('PMID'),'literature':len(i.get('PMID')),'targetshapes':i.get('targetshapes'),'allyears':i.get('allyears'),'type':i.get('type'),'MeshID':i.get('MESH'),'Summary':'','url':'https://www.ncbi.nlm.nih.gov'+i.get('url'),'_id':i.get('type')+'_'+i.get('entity'),'datest':int(i.get('datestring')[0].replace('-',''))})
for i in Toxin_all:
    if i.get('Summary'):
        all_es.append({'entity':i.get('entity'),'PMID':i.get('PMID'),'literature':len(i.get('PMID')),'targetshapes':i.get('targetshapes'),'allyears':i.get('allyears'),'type':i.get('type'),'MeshID':i.get('MESH'),'Summary':sent_tokenize(i.get('Summary'))[0],'url':'https://www.ncbi.nlm.nih.gov'+i.get('url'),'_id':i.get('type')+'_'+i.get('entity'),'datest':int(i.get('datestring')[0].replace('-',''))})
    else:
        all_es.append({'entity':i.get('entity'),'PMID':i.get('PMID'),'literature':len(i.get('PMID')),'targetshapes':i.get('targetshapes'),'allyears':i.get('allyears'),'type':i.get('type'),'MeshID':i.get('MESH'),'Summary':'','url':'https://www.ncbi.nlm.nih.gov'+i.get('url'),'_id':i.get('type')+'_'+i.get('entity'),'datest':int(i.get('datestring')[0].replace('-',''))})
for i in Mutation_all:
    if i.get('Genomic View'):
        all_es.append({'entity':i.get('entity'),'PMID':i.get('PMID'),'literature':len(i.get('PMID')),'targetshapes':i.get('targetshapes'),'allyears':i.get('allyears'),'type':i.get('type'),'Genomic_View':i.get('Genomic View'),'Alleles':i.get('Alleles'),'Position':i.get('Position'),'_id':i.get('type')+'_'+i.get('entity'),'datest':int(i.get('datestring')[0].replace('-',''))})
    else:
        mu+=1
for i in Disease_all:
    if i.get('url'):
        if i.get('Summary'):
            all_es.append({'entity':i.get('entity'),'PMID':i.get('PMID'),'literature':len(i.get('PMID')),'targetshapes':i.get('targetshapes'),'allyears':i.get('allyears'),'type':i.get('type'),'MeshID':i.get('MESH'),'url':'https://www.ncbi.nlm.nih.gov'+i.get('url'),'Summary':sent_tokenize(i.get('Summary'))[0],'_id':i.get('type')+'_'+i.get('entity'),'datest':int(i.get('datestring')[0].replace('-',''))})
        else:
            all_es.append({'entity':i.get('entity'),'PMID':i.get('PMID'),'literature':len(i.get('PMID')),'targetshapes':i.get('targetshapes'),'allyears':i.get('allyears'),'type':i.get('type'),'MeshID':i.get('MESH'),'url':'https://www.ncbi.nlm.nih.gov'+i.get('url'),'Summary':'','_id':i.get('type')+'_'+i.get('entity'),'datest':int(i.get('datestring')[0].replace('-',''))})
    else:
        di+=1
        noturl.append(i)

In [613]:
all_es_copy=copy.deepcopy(all_es)

In [614]:
#10907
len(all_es_copy)

10907

In [615]:
for i in all_es:
    i['allwordsdict']={}
    if i.get('Also_known_as'):
        i['allwords']=i.get('allwords',[])+i.get('Also_known_as').split('; ')
        for l in i.get('Also_known_as').split('; '):
            i['allwordsdict'][l.upper()]=l
    for j in i.get('targetshapes'):
        if j.upper() != i.get('entity').upper() and not i.get('allwordsdict').get(j.upper()):
            i['allwords']=i.get('allwords',[])+[j.upper()]
            i['allwordsdict'][j.upper()]=j
    if i.get('targetshapes'):
        del i['targetshapes']
    del i['allwordsdict']

In [616]:
all_es[-1000]

{'entity': 'Abnormalities, Multiple',
 'PMID': ['30612976',
  '29488253',
  '25514088',
  '16207588',
  '15901576',
  '15613268',
  '11207836',
  '7844065',
  '1612301'],
 'literature': 9,
 'allyears': [2019, 2018, 2015, 2005, 2004, 2001, 1994, 1992],
 'type': 'Disease',
 'MeshID': 'D000015',
 'url': 'https://www.ncbi.nlm.nih.gov/mesh/68000015',
 'Summary': 'Congenital abnormalities that affect more than one organ or body structure.',
 '_id': 'Disease_Abnormalities, Multiple',
 'datest': 20190701,
 'allwords': ['NERVOUS ABNORMALITY',
  'MULTIPLE VISUAL DEFECTS',
  'MULTIPLE ABNORMALITIES']}

In [617]:
len(all_es)

10907

In [618]:
noturls=[]
for i in Disease_all:
    if not i.get('url'):
        noturls.append(i)

In [619]:
#13986
len(noturls)

13986

In [620]:
# 17330
len(Disease_all)

17330

In [621]:
for i in all_es:
    if i.get('entity')=='Death':
        print(i.get('allwords')[:20])
        break

['SEPSIS DEATH', 'HEAD INJURY AND ACCIDENTAL DEATH', 'DIED OF HEART DISEASE', 'NEUROLOGICAL DISEASE DEATHS', 'BRAIN METASTASES DIED', 'DECREASED HEMORRHAGIC DEATHS', 'STROKE DIED', 'CANCER-MORTALITY', 'DI PALMA', 'TRAUMA DEATH', 'DETERMINATION OF FAILURE', 'DEATH HYPOGLYCEMIC SYNCOPE', 'COLORECTAL MORTALITY', 'LATE DEATHS', 'MORTALITY OF EPILEPSY', 'DIED OF MALIGNANT DISEASE', 'DIED OF RENAL DISEASE', 'DI ONCOLOGIA', 'INJURIES DIED', 'CARCINOMA DEATH']


In [622]:
len(all_es)

10907

for i in Disease_all:
    if i.get('entity')=='Parkinson Disease':
        for j,k in enumerate(i.get('target')):
            for l in k:
                if "ALZHEI-MER'S DISEASE" == l.upper():
                    print(i.get('PMID')[j])
                    break

for i in all_es:
    if i.get('allwords') and "ALZHEI-MER'S DISEASE" in i.get('allwords'):
        # print(i.get('PMID'))
        print(i.get('entity'))
        print(i.get('allwords').index("ALZHEI-MER'S DISEASE"))
        print(i.get('PMID')[i.get('allwords').index("ALZHEI-MER'S DISEASE")])


In [623]:
json.dump(all_es,open('results/all_es_lack.json','w'))

In [627]:
entity_pairs=['Gene','RNA','Carbohydrate','Lipid','Peptide', 'Pharmaceutical_Preparations', 'Protein', 'Toxin','Mutation', 'Disease']

In [628]:
all_es_result=[]
for j in entity_pairs:
    for i in names[j+'_all']:
        if i.get('specific_type'):
            all_es_result.append({'entity':i.get('entity'),'PMID':i.get('PMID'),'target':i.get('target'),'date':i.get('date'),'type':i.get('specific_type'),'sentence':i.get('sentence'),'_id':i.get('type')+'_'+i.get('entity')})
        else:
            if i.get('type')=='Disease':
                if i.get('url'):
                    all_es_result.append({'entity':i.get('entity'),'PMID':i.get('PMID'),'target':i.get('target'),'date':i.get('date'),'type':i.get('type'),'sentence':i.get('sentence'),'_id':i.get('type')+'_'+i.get('entity')})
            elif i.get('type')=='Mutation':
                if i.get('Genomic View'):
                    all_es_result.append({'entity':i.get('entity'),'PMID':i.get('PMID'),'target':i.get('target'),'date':i.get('date'),'type':i.get('type'),'sentence':i.get('sentence'),'_id':i.get('type')+'_'+i.get('entity')})
            else:
                all_es_result.append({'entity':i.get('entity'),'PMID':i.get('PMID'),'target':i.get('target'),'date':i.get('date'),'type':i.get('type'),'sentence':i.get('sentence'),'_id':i.get('type')+'_'+i.get('entity')})

In [629]:
jcr=pd.read_csv("../JCR/jcr2022.tsv",sep="\t")

In [630]:
collections_pubtator_info = db.pubtator_info

In [631]:
pubtator_info=list(collections_pubtator_info.find())

In [632]:
pubtator_info_dict={}
for i in pubtator_info:
    pubtator_info_dict[i.get('PMID')]=i

In [633]:
jt2if_all={}
for i in range(len(jcr)):
    if jt2if_all.get(jcr.iloc[i,1].lower()):
        if jt2if_all.get(jcr.iloc[i,1].lower())==0.0:
            jt2if_all[jcr.iloc[i,1].lower()]=jcr.iloc[i,7]
    else:
        jt2if_all[jcr.iloc[i,1].lower()]=jcr.iloc[i,7]
jt2if_all['aging']=jt2if_all.get('aging-us')

In [634]:
jt2if_brief={}
for i in range(len(jcr)):
    if jt2if_brief.get(jcr.iloc[i,2].lower()):
        if jt2if_brief.get(jcr.iloc[i,2].lower())==0.0:
            jt2if_brief[jcr.iloc[i,2].lower()]=jcr.iloc[i,7]
    else:
        jt2if_brief[jcr.iloc[i,2].lower()]=jcr.iloc[i,7]
jt2if_brief['aging']=jt2if_brief.get('aging-us')

In [635]:
jt2if5_all={}
for i in range(len(jcr)):
    if jt2if5_all.get(jcr.iloc[i,1].lower()):
        if jt2if5_all.get(jcr.iloc[i,1].lower())==0.0:
            jt2if5_all[jcr.iloc[i,1].lower()]=jcr.iloc[i,7]
    else:
        jt2if5_all[jcr.iloc[i,1].lower()]=jcr.iloc[i,9]
jt2if5_all['aging']=jt2if5_all.get('aging-us')

In [636]:
jt2if5_brief={}
for i in range(len(jcr)):
    
    if jt2if5_brief.get(jcr.iloc[i,2].lower()):
        if jt2if5_brief.get(jcr.iloc[i,2].lower())==0.0:
            jt2if5_brief[jcr.iloc[i,2].lower()]=jcr.iloc[i,7]
    else:
        jt2if5_brief[jcr.iloc[i,2].lower()]=jcr.iloc[i,9]
jt2if5_brief['aging']=jt2if5_brief.get('aging-us')

        if jt2if_brief.get(pubtator_info_dict.get(j).get('TA').lower()):
            iflist.append(jt2if_brief.get(pubtator_info_dict.get(j).get('TA').lower()))
        elif jt2if_all.get(pubtator_info_dict.get(j).get('JT').lower()):
            iflist.append(jt2if_all.get(pubtator_info_dict.get(j).get('JT').lower()))
        else:
            iflist.append(0.0)
        if jt2if5_brief.get(pubtator_info_dict.get(j).get('TA').lower()):
            if5list.append(jt2if5_brief.get(pubtator_info_dict.get(j).get('TA').lower()))
        elif jt2if5_all.get(pubtator_info_dict.get(j).get('JT').lower()):
            if5list.append(jt2if5_all.get(pubtator_info_dict.get(j).get('JT').lower()))
        else:
            if5list.append(0.0)

In [637]:
for i in all_es_result:
    iflist=[]
    if5list=[]
    TAlist=[]
    JTlist=[]
    yearlist=[]
    for j in i.get('PMID'):
        if jt2if_brief.get(pubtator_info_dict.get(j).get('TA').lower()):
            iflist.append(jt2if_brief.get(pubtator_info_dict.get(j).get('TA').lower()))
        elif jt2if_all.get(pubtator_info_dict.get(j).get('JT').lower()):
            iflist.append(jt2if_all.get(pubtator_info_dict.get(j).get('JT').lower()))
        else:
            iflist.append(0.0)
        if jt2if5_brief.get(pubtator_info_dict.get(j).get('TA').lower()):
            if5list.append(jt2if5_brief.get(pubtator_info_dict.get(j).get('TA').lower()))
        elif jt2if5_all.get(pubtator_info_dict.get(j).get('JT').lower()):
            if5list.append(jt2if5_all.get(pubtator_info_dict.get(j).get('JT').lower()))
        else:
            if5list.append(0.0)
        TAlist.append(pubtator_info_dict.get(j).get('TA'))
        JTlist.append(pubtator_info_dict.get(j).get('JT'))
    for l in i.get('date'):
        yearlist.append(int(str(l)[:4]))
    i['IF']=iflist
    i['IF5']=if5list
    i['TA']=TAlist
    i['JT']=JTlist
    i['year']=yearlist

In [638]:
json.dump(all_es_result,open('results/all_es_result.json','w'))

In [639]:
#10907
len(all_es_result)

10907

In [ ]:
Pharmaceutical_Preparations_all[0].get('type')

In [ ]:
pwd

In [1029]:
blank=[{'Module':'','Description':'','Entity_Relation':'','Username':'','Email':''}]

In [1030]:
json.dump(blank,open('feedback.json','w'))

In [322]:
len(RNA2approved_dict)

8883

In [323]:
i

'MIR29A'

In [140]:
import http.client
print(http.client.HTTPConnection._http_vsn)
print(http.client.HTTPConnection._http_vsn_str)

11
HTTP/1.1


list(db.Disease_raw.find().limit(1))

list(db.Peptide_raw.find().limit(3))

list(db.Gene_raw.find().limit(1))

list(db.RNA_raw.find().limit(1))

list(db.Carbohydrate_raw.find().limit(1))

list(db.Lipid_raw.find().limit(1))

list(db.Mutation_raw.find().limit(1))

list(db.Disease_raw.find().limit(1))

db.Disease_raw.remove({'addtime':now_time})

db.Carbohydrate_raw.remove({'addtime':now_time})

db.Lipid_raw.remove({'addtime':now_time})

db.Peptide_raw.remove({'addtime':now_time})

db.Pharmaceutical_Preparations_raw.remove({'addtime':now_time})

db.Protein_raw.remove({'addtime':now_time})

db.Toxin_raw.remove({'addtime':now_time})

In [372]:
Gene_entity_blank=[]
for i in Gene_all:
    Gene_entity_blank.append(i.get('entity'))

In [373]:
len(Gene_entity_blank)

4854

In [395]:
len(set(Gene_entity_blank))

4854

In [453]:
import time
import datetime
from multiprocessing import Process, Manager, Pool
import os, time, random
from lxml import etree
import requests
def get_Gene_info(i,final_Genes_inte):
    ncbiid=Gene2ncbi_dict.get(i)
    ncbiurl='https://www.ncbi.nlm.nih.gov/gene/' + ncbiid
    headers = {
        "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 \
        (KHTML, like Gecko) Chrome/83.0.4103.97 Safari/537.36"
    }
    url = 'https://www.ncbi.nlm.nih.gov/gene/'+ncbiid
    
    res=requests.get(url=url,headers=headers)
    html = etree.HTML(res.content)
    ncbisummary={
    # 'numbersOfLiterature' : len(i.get('PMID')),
    'Official_Symbol' : html.xpath('//*[@id="summaryDl"]/dt[contains(.//text(),"Official")]/following::dd[1]/text()')[0],
    'Official_Full_Name' : html.xpath('//*[@id="summaryDl"]/dt[contains(.//text(),"Official")]/following::dd[1]/text()')[1],
    # 'Also_known_as' : ''.join(html.xpath('//*[@id="summaryDl"]/dt[contains(.//text(),"Also known as")]/following::dd[1]/text()')).replace('HGNC:HGNC:',''),
    'HGNC_ID' : ''.join(html.xpath('//*[@id="summaryDl"]/dt[contains(.//text(),"Primary source")]/following::dd[1]/a/text()')),
    'HGNC_URL' : ''.join(html.xpath('//*[@id="summaryDl"]/dt[contains(.//text(),"Primary source")]/following::dd[1]/a/@href')),
    'Ensembl_ID' :  ''.join(html.xpath('//*[@id="summaryDl"]/dt[contains(.//text(),"See related")]/following::dd[1]/a[1]/text()')),
    'Ensembl_URL' : ''.join(html.xpath('//*[@id="summaryDl"]/dt[contains(.//text(),"See related")]/following::dd[1]/a[1]/@href')),
    'MIM_ID' : ''.join(html.xpath('//*[@id="summaryDl"]/dt[contains(.//text(),"See related")]/following::dd[1]/a[2]/text()')),
    'MIM_URL' : ''.join(html.xpath('//*[@id="summaryDl"]/dt[contains(.//text(),"See related")]/following::dd[1]/a[2]/@href')),
    'AllianceGenome_ID' : ''.join(html.xpath('//*[@id="summaryDl"]/dt[contains(.//text(),"See related")]/following::dd[1]/a[3]/text()')),
    'AllianceGenome_URL' : ''.join(html.xpath('//*[@id="summaryDl"]/dt[contains(.//text(),"See related")]/following::dd[1]/a[3]/@href')),
    'NCBI_ID' : ncbiid ,
    'NCBI_URL' : ncbiurl
    }
    try:
        also=''.join(html.xpath('//*[@id="summaryDl"]/dt[contains(.//text(),"Also known as")]/following::dd[1]/text()')).replace('HGNC:HGNC:','')
    except:
        also=''
    finally:
        ncbisummary.update({'Also_known_as' : also})
    try:
        also2=''.join(html.xpath('//*[@id="summaryDl"]/dt[contains(.//text(),"Summary")]/following::dd[1]/text()'))
    except:
        also2=''
    finally:
        ncbisummary.update({'Summary' : also2})
    final_Genes_inte[i]=ncbisummary
    # print(ncbisummary)
    time.sleep(0.5)

get_Gene_info(i)

In [454]:
p = Pool(10)
manager=Manager()
final_Genes_inte = manager.dict()
for i in Gene_entity_blank:
    p.apply_async(get_Gene_info, args=(i,final_Genes_inte))
p.close()
p.join()

In [455]:
len(dict(final_Genes_inte))

4854

In [466]:
Gene_converter=dict(final_Genes_inte)

In [457]:
pickle.dump(dict(final_Genes_inte), open('../ncbi/Gene_converter.pkl','wb'))

In [376]:
RNA_entity_blank=[]
for i in RNA_all:
    RNA_entity_blank.append(i.get('entity'))

In [377]:
len(RNA_entity_blank)

339

In [459]:
import time
import datetime
from multiprocessing import Process, Manager, Pool
import os, time, random
from lxml import etree
import requests
def get_RNA_info(i,final_RNAs_inte):
    ncbiid=RNA2ncbi_dict.get(i)
    ncbiurl='https://www.ncbi.nlm.nih.gov/gene/' + ncbiid
    headers = {
        "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 \
        (KHTML, like Gecko) Chrome/83.0.4103.97 Safari/537.36"
    }
    url = 'https://www.ncbi.nlm.nih.gov/gene/'+ncbiid
    
    res=requests.get(url=url,headers=headers)
    html = etree.HTML(res.content)
    ncbisummary={
    # 'numbersOfLiterature' : len(i.get('PMID')),
    'Official_Symbol' : html.xpath('//*[@id="summaryDl"]/dt[contains(.//text(),"Official")]/following::dd[1]/text()')[0],
    'Official_Full_Name' : html.xpath('//*[@id="summaryDl"]/dt[contains(.//text(),"Official")]/following::dd[1]/text()')[1],
    # 'Also_known_as' : ''.join(html.xpath('//*[@id="summaryDl"]/dt[contains(.//text(),"Also known as")]/following::dd[1]/text()')).replace('HGNC:HGNC:',''),
    'HGNC_ID' : ''.join(html.xpath('//*[@id="summaryDl"]/dt[contains(.//text(),"Primary source")]/following::dd[1]/a/text()')),
    'HGNC_URL' : ''.join(html.xpath('//*[@id="summaryDl"]/dt[contains(.//text(),"Primary source")]/following::dd[1]/a/@href')),
    'Ensembl_ID' :  ''.join(html.xpath('//*[@id="summaryDl"]/dt[contains(.//text(),"See related")]/following::dd[1]/a[1]/text()')),
    'Ensembl_URL' : ''.join(html.xpath('//*[@id="summaryDl"]/dt[contains(.//text(),"See related")]/following::dd[1]/a[1]/@href')),
    'MIM_ID' : ''.join(html.xpath('//*[@id="summaryDl"]/dt[contains(.//text(),"See related")]/following::dd[1]/a[2]/text()')),
    'MIM_URL' : ''.join(html.xpath('//*[@id="summaryDl"]/dt[contains(.//text(),"See related")]/following::dd[1]/a[2]/@href')),
    'AllianceGenome_ID' : ''.join(html.xpath('//*[@id="summaryDl"]/dt[contains(.//text(),"See related")]/following::dd[1]/a[3]/text()')),
    'AllianceGenome_URL' : ''.join(html.xpath('//*[@id="summaryDl"]/dt[contains(.//text(),"See related")]/following::dd[1]/a[3]/@href')),
    'NCBI_ID' : ncbiid ,
    'NCBI_URL' : ncbiurl
    }
    try:
        also=''.join(html.xpath('//*[@id="summaryDl"]/dt[contains(.//text(),"Also known as")]/following::dd[1]/text()')).replace('HGNC:HGNC:','')
    except:
        also=''
    finally:
        ncbisummary.update({'Also_known_as' : also})
    try:
        also2=''.join(html.xpath('//*[@id="summaryDl"]/dt[contains(.//text(),"Summary")]/following::dd[1]/text()'))
    except:
        also2=''
    finally:
        ncbisummary.update({'Summary' : also2})
    final_RNAs_inte[i]=ncbisummary
    # print(ncbisummary)
    time.sleep(0.5)

In [460]:
p = Pool(10)
manager=Manager()
final_RNAs_inte = manager.dict()
for i in RNA_entity_blank:
    p.apply_async(get_RNA_info, args=(i,final_RNAs_inte))
p.close()
p.join()

In [461]:
len(dict(final_RNAs_inte))

339

In [467]:
RNA_converter=dict(final_RNAs_inte)

In [462]:
list(dict(final_RNAs_inte).items())[:2]

[('MIR30A',
  {'Official_Symbol': 'MIR30A',
   'Official_Full_Name': 'microRNA 30a',
   'HGNC_ID': 'HGNC:HGNC:31624',
   'HGNC_URL': 'https://www.genenames.org/data/gene-symbol-report/#!/hgnc_id/HGNC:31624',
   'Ensembl_ID': 'Ensembl:ENSG00000207827',
   'Ensembl_URL': 'http://www.ensembl.org/id/ENSG00000207827',
   'MIM_ID': 'MIM:612329;',
   'MIM_URL': 'https://www.ncbi.nlm.nih.gov/omim/612329',
   'AllianceGenome_ID': 'miRBase:MI0000088;',
   'AllianceGenome_URL': 'http://www.mirbase.org/cgi-bin/mirna_entry.pl?acc=MI0000088',
   'NCBI_ID': '407029',
   'NCBI_URL': 'https://www.ncbi.nlm.nih.gov/gene/407029',
   'Also_known_as': 'MIRN30A; mir-30a',
   'Summary': 'microRNAs (miRNAs) are short (20-24 nt) non-coding RNAs that are involved in post-transcriptional regulation of gene expression in multicellular organisms by affecting both the stability and translation of mRNAs. miRNAs are transcribed by RNA polymerase II as part of capped and polyadenylated primary transcripts (pri-miRNAs) 

In [463]:
pickle.dump(dict(final_RNAs_inte), open('../ncbi/RNA_converter.pkl','wb'))

In [950]:
pwd

'/root/hald_autoupdate/2022_12_09'

In [951]:
pairs

['Toxin',
 'Peptide',
 'Lipid',
 'Protein',
 'Carbohydrate',
 'Pharmaceutical Preparations']

In [956]:
all_es_result[10000]

{'entity': 'Amino Acid Metabolism, Inborn Errors',
 'PMID': ['33742102',
  '20564567',
  '12052660',
  '10563973',
  '7676443',
  '35527334'],
 'target': [['inborn errors of amino acid metabolism'],
  ['cross-linked amino acids'],
  ['excitotoxic amino acids'],
  ['cross-linked amino acids'],
  ['excitotoxic amino acids'],
  ['amino acid metabolism']],
 'date': [20210319, 20100601, 20020228, 19990401, 19950901, 20220701],
 'type': 'Disease',
 'sentence': [['They represent a general class of metabolites for inborn errors of amino acid metabolism that show potential as biomarkers and may provide further insight in disease pathophysiology.'],
  ['Exposure of food proteins to certain processing conditions induces two major chemical changes: racemization of all L-amino acids (LAAs) to D-amino acids (DAAs) and concurrent formation of cross-linked amino acids such as lysinoalanine (LAL).'],
  ['Both in vivo and in culture, nicotine protects striatal, hippocampal and cortical neurons against t

In [955]:
for i in all_es_copy:
    if i.get('type')=='Pharmaceutical Preparations':
        print('w')
        break

w


In [968]:
for i in all_es_result:
    if i.get('type')=='microRNA':
        print('i')
        break

In [953]:
entity_pairs

['DNA',
 'RNA',
 'Carbohydrate',
 'Lipid',
 'Peptide',
 'Pharmaceutical_Preparations',
 'Protein',
 'Toxin',
 'Mutation',
 'Disease']

In [640]:
len(all_es_result)

10907

In [641]:
for i in entity_pairs:
    names[i+'_literature']=set()
    names[i+'_sentence']=set()

for j in entity_pairs:
    for i in all_es_result:
        if i.get('type')==j.replace('_',' '):
            for k in i.get('PMID'):
                names[j+'_literature'].add(k)
            for l in i.get('sentence'):
                for m in l:
                    names[j+'_sentence'].add(m)
            continue
        elif i.get('type')[-3:]=='RNA':
            for k in i.get('PMID'):
                names[j+'_literature'].add(k)
            for l in i.get('sentence'):
                for m in l:
                    names[j+'_sentence'].add(m)

In [642]:
for i in entity_pairs:
    print(i+' '+str(len(list(names[i+'_literature']))))

Gene 25896
RNA 643
Carbohydrate 8484
Lipid 7862
Peptide 2200
Pharmaceutical_Preparations 790
Protein 1350
Toxin 908
Mutation 2073
Disease 145981


In [1003]:
for i in entity_pairs:
    print(i+' '+str(len(list(names[i+'_literature']))))

DNA 25896
RNA 643
Carbohydrate 8484
Lipid 7861
Peptide 2199
Pharmaceutical_Preparations 790
Protein 1350
Toxin 907
Mutation 2617
Disease 145981


In [643]:
for i in entity_pairs:
    print(i+' '+str(len(list(names[i+'_sentence']))))

Gene 78418
RNA 1986
Carbohydrate 17462
Lipid 16921
Peptide 5035
Pharmaceutical_Preparations 2222
Protein 3677
Toxin 2438
Mutation 5345
Disease 619831


In [995]:
nnn=0
for i in all_es_result:
    if i.get('type')=='Disease':
        nnn+=1
print(nnn)

17330


In [1012]:
len(all_es_result)

10907

In [1009]:
len(all_es)

10907

In [963]:
for i in entity_pairs:
    print(i+' '+str(len(list(names[i+'_literature']))))

DNA 25615
RNA 0
Carbohydrate 7886
Lipid 7250
Peptide 1558
Pharmaceutical_Preparations 147
Protein 709
Toxin 270
Mutation 2005
Disease 156964


In [960]:
len(list(Gene_literature))

25615

In [961]:
len(list(Gene_sentence))

77013

In [644]:
pwd

'/root/hald_autoupdate/2022_12_09'